***
# PheKnowLator - Data Preparation
***
***

**Author:** [TJCallahan](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=callahantiff@gmail.com)  
**GitHub Repository:** [PheKnowLator](https://github.com/callahantiff/PheKnowLator/wiki)  
**Release:** **[v2.0.0](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**
  
<br>  
  
**Purpose:** This notebook serves as a script to download and process data in order to generate mapping and filtering data needed to build edges for the PheKnowLator knowledge graph. For more information on the data sources utilize within this script, please see the [Data Sources](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources) Wiki page.

<br>

**Assumptions:**   
- Raw data downloads ➞ `./resources/processed_data/unprocessed_data`    
- Processed data write location ➞ `./resources/processed_data`  

<br>

**Dependencies:** This notebook utilizes several helper functions, which are stored in the [`data_preparation_helper_functions.py`](https://github.com/callahantiff/PheKnowLator/blob/master/scripts/python/data_preparation_helper_functions.py) script. Hyperlinks to all downloaded and generated data sources are provided on the [Data Sources](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources) Wiki page as well as within each source subsection of this notebook. All generated data is freely available for download from DropBox. 

_____
***

## Table of Contents
***

### [Create Identifier Maps ](#create-identifier-maps)  
- [HUMAN TRANSCRIPT, GENE, AND PROTEIN IDENTIFIER MAPPING](#human-transcript,-gene,-and-protein-identifier-mapping)
  - [Entrez Gene-Ensembl Transcript](#entrezgene-ensembltranscript)  
  - [Entrez Gene-Protein Ontology](#entrezgene-proteinontology)  
  - [Ensembl Gene-Entrez Gene](#ensemblgene-entrezgene)
  - [Gene Symbol-Ensembl Transcript](#genesymbol-ensembltranscript)  
  - [STRING-Protein Ontology](#string-proteinontology)  
  - [Uniprot Accession-Protein Ontology](#uniprotaccession-proteinontology)
  

- [OTHER IDENTIFIER MAPPING](#other-identifier-mapping) 
  - [ChEBI Identifiers](#mesh-chebi) 
  - [Human Disease and Phenotype Identifiers](#disease-identifiers)
  - [Human Protein Atlas Tissue and Cell Types](#hpa-uberon)  

<br>

### [Create Edge Datasets](#create-edge-datasets)
- [ONTOLOGIES](#ontologies)  
  - [Protein Ontology](#protein-ontology)  
  - [Relations Ontology](#relations-ontology)  


- [LINKED DATA](#linked-data)  
  - [Clinvar Variant-Diseases and Phenotypes](#clinvar-variant)
  - [Reactome Chemical-Complex Data](#reactome-chemical-complex)
  - [Reactome Complex-Complex Data](#reactome-complex-complex)
  - [Reactome Complex-Pathway Data](#reactome-complex-pathway)
  - [Reactome Protein-Complex Data](#reactome-protein-complex)
  - [Uniprot Protein-Cofactor and Protein-Catalyst](#uniprot-protein-cofactorcatalyst)  

<br>

### [Create Instance Data Metadata](#create-instance-metadata)  
- [Genes/RNA](#gene-and-rna-metadata)
- [Pathways](#pathway-metadata)
- [Complexes](#complex-metadata)
- [Reactions](#reaction-metadata)
- [Variants](#variant-metadata) 

____

<br>

### Set-Up Environment
_____

In [ ]:
# import needed libraries
import datetime
import glob
import itertools
import networkx
import numpy
import pandas

from collections import Counter
from functools import reduce
from owlready2 import subprocess
from rdflib import Graph, Namespace, URIRef, BNode, extras, Literal
from rdflib.extras.external_graph_libs import *
from reactome2py import content
from tqdm import tqdm

# import script containing helper functions
from scripts.python.data_preparation_helper_functions import *

**Define Global Variables**

In [ ]:
# directory to read unprocessed data files from
unprocessed_data_location = 'resources/processed_data/unprocessed_data/'

# directory to write processed data files to
processed_data_location = 'resources/processed_data/'

# directory to write relations data to
relations_data_location = 'resources/relations_data/'

# directory to write node metadata to
node_data_location = 'resources/node_data/'

<br>

***
***
### CREATE MAPPING DATASETS  <a class="anchor" id="create-identifier-maps"></a>
***
***

### Human Transcript, Gene, and Protein Identifier Mapping  <a class="anchor" id="human-transcript,-gene,-and-protein-identifier-mapping"></a>
***

**Data Source Wiki Pages:**   
- [Ensembl](https://uswest.ensembl.org/)  
- [Uniprot Knowledgebase](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#uniprot-knowledgebase)  
- [HGNC](ftp://ftp.ebi.ac.uk/pub/databases/genenames/new/tsv/hgnc_complete_set.txt) 
- [NCBI Gene](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#ncbi-gene) 
- [Protein Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#protein-ontology)

<br>

**Purpose:** To map create protein-coding gene-protein relations and mappings between the identifiers listed below. The edges types produced from each of these mappings will be further described within each identifier mapping section:  
- [Entrez Gene-Ensembl Transcript](#entrezgene-ensembltranscript)  
- [Entrez Gene-Protein Ontology](#entrezgene-proteinontology)  
- [Ensembl Gene-Entrez Gene](#ensemblgene-entrezgene)
- [Gene Symbol-Ensembl Transcript](#genesymbol-ensembltranscript)  
- [STRING-Protein Ontology](#string-proteinontology)  
- [Uniprot Accession-Protein Ontology](#uniprotaccession-proteinontology)

<br>

**Gene and Trainscript Types:** The transcript and gene/locus types were reviewed by a PhD Molecular biologist to confirm whether or not they should be treated as `protein-coding` or not, which is useful for creating `genomic-rna`, `genomic-protein`, and `rna-protein` edges in the knowledge graph. For more information on this classification, please see the table below. Definitions of concepts in the table have been taken from [HGNC](https://www.genenames.org/help/symbol-report/), [Ensembl](https://uswest.ensembl.org/info/genome/genebuild/biotypes.html), [NCBI](https://www.ncbi.nlm.nih.gov/IEB/ToolBox/CPP_DOC/lxr/source/src/objects/entrezgene/entrezgene.asn), and Wikipedia.

Gene and Transcript Type | Definition | Type | Genomic material Transcribed to RNA | RNA Translated to Protein | Genomic material has_gene_product Protein
-- | -- | -- | -- | -- | --
biological-region | biological_region (SO:0001411; Special note: This is a parental feature spanning all other feature annotation on each RefSeq Functional Element record. It is a 'misc_feature' in GenBank flat files but a 'Region' feature in ASN.1 and GFF3 formats | gene | yes | no | no
IG_C_gene | IG C gene: Constant chain immunoglobulin gene that undergoes somatic recombination before transcription | gene | yes | no | no
IG_C_gene | IG C gene: Constant chain immunoglobulin gene that undergoes somatic recombination before transcription | transcript | yes | yes | no
IG_C_pseudogene | Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame | gene | yes | no | no
IG_C_pseudogene | Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame | transcript | yes | yes | no
IG_D_gene | IG D gene: Diversity chain immunoglobulin gene that undergoes somatic recombination before transcription | gene | yes | no | no
IG_D_gene | IG D gene: Diversity chain immunoglobulin gene that undergoes somatic recombination before transcription | transcript | yes | yes | no
IG_J_gene | IG J gene: Joining chain immunoglobulin gene that undergoes somatic recombination before transcription | gene | yes | no | no
IG_J_gene | IG J gene: Joining chain immunoglobulin gene that undergoes somatic recombination before transcription | transcript | yes | yes | no
IG_J_pseudogene | Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame | gene | yes | no | no
IG_J_pseudogene | Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame | transcript | yes | yes | no
IG_pseudogene | Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame | gene | yes | no | no
IG_pseudogene | Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame | transcript | yes | yes | no
IG_V_gene | IG V gene: Variable chain immunoglobulin gene that undergoes somatic recombination before transcription | gene | yes | no | no
IG_V_gene | IG V gene: Variable chain immunoglobulin gene that undergoes somatic recombination before transcription | transcript | yes | yes | no
IG_V_pseudogene | Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame | gene | yes | no | no
IG_V_pseudogene | Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame | transcript | yes | yes | no
lncRNA | RNA, long non-coding - non-protein coding genes that encode long non-coding RNAs (lncRNAs) (SO:0001877); these are at least 200 nt in length. Subtypes include intergenic (SO:0001463), intronic (SO:0001903) and antisense (SO:0001904) | gene | yes | no | no
lncRNA | RNA, long non-coding - non-protein coding genes that encode long non-coding RNAs (lncRNAs) (SO:0001877); these are at least 200 nt in length. Subtypes include intergenic (SO:0001463), intronic (SO:0001903) and antisense (SO:0001904) | transcript | yes | no | no
miRNA | RNA, micro - non-protein coding genes that encode microRNAs (miRNAs) (SO:0001265) | gene | yes | no | no
miRNA | RNA, micro - non-protein coding genes that encode microRNAs (miRNAs) (SO:0001265) | transcript | yes | no | no
misc_RNA | non-protein coding genes that encode miscellaneous types of small ncRNAs, such as vault (SO:0000404) and Y (SO:0000405) RNA genes | gene | yes | no | no
misc_RNA | non-protein coding genes that encode miscellaneous types of small ncRNAs, such as vault (SO:0000404) and Y (SO:0000405) RNA genes | transcript | yes | no | no
Mt_rRNA | mitochondrial rRNA | gene | yes | no | no
Mt_rRNA | mitochondrial rRNA | transcript | yes | no | no
Mt_tRNA | mitochondrial tRNA | gene | yes | no | no
Mt_tRNA | mitochondrial tRNA | transcript | yes | no | no
ncRNA | Noncoding RNA | gene | yes | no | no
non_stop_decay | Nonstop decay. Transcripts that have polyA features (including signal) without a prior stop codon in the CDS, i.e. a non-genomic polyA tail attached directly to the CDS without 3' UTR. These transcripts are subject to degradation. | transcript | yes | no | no
None | None | gene | yes | no | no
None | None | transcript | yes | no | no
nonsense_mediated_decay | Nonsense mediated decay. If the coding sequence (following the appropriate reference) of a transcript finishes >50bp from a downstream splice site then it is tagged as NMD. If the variant does not cover the full reference coding sequence then it is annotated as NMD if NMD is unavoidable i.e. no matter what the exon structure of the missing portion is the transcript will be subject to NMD | transcript | yes | yes | no
other | Other | gene | yes | no | no
phenotype | --- | gene | yes | no | no
polymorphic_pseudogene | Polymorphic pseudogene. Pseudogene owing to a SNP/DIP but in other individuals/haplotypes/strains the gene is translated | gene | yes | no | no
polymorphic_pseudogene | Polymorphic pseudogene. Pseudogene owing to a SNP/DIP but in other individuals/haplotypes/strains the gene is translated | transcript | yes | yes | no
processed_pseudogene | Processed pseudogene. Pseudogene that lack introns and is thought to arise from reverse transcription of mRNA followed by reinsertion of DNA into the genome | gene | yes | no | no
processed_pseudogene | Processed pseudogene. Pseudogene that lack introns and is thought to arise from reverse transcription of mRNA followed by reinsertion of DNA into the genome | transcript | yes | yes | no
processed_transcript | Processed transcript: Gene/transcript that doesn't contain an open reading frame | transcript | yes | yes | no
protein_coding | Protein coding. Contains an open reading frame (ORF) | transcript | yes | yes | yes
protein-coding | Protein coding. Contains an open reading frame (ORF) | gene | yes | no | yes
pseudogene | Similar to known proteins but contain a frameshift and/or stop codon(s) which disrupts the ORF | gene | yes | no | no
pseudogene | Pseudogene. Have homology to proteins but generally suffer from a disrupted coding sequence and an active homologous gene can be found at another locus. Sometimes these entries have an intact coding sequence or an open but truncated ORF, in which case there is other evidence used (for example genomic polyA stretches at the 3' end) to classify them as a pseudogene | transcript | yes | yes | no
retained_intron | Retained_intron. Has an alternatively spliced transcript believed to contain intronic sequence relative to other, coding, variants | transcript | yes | yes | no
ribozyme | Ribozymes are RNA molecules that have the ability to catalyze specific biochemical reactions, including RNA splicing in gene expression, similar to the action of protein enzymes | gene | yes | no | no
ribozyme | Ribozymes are RNA molecules that have the ability to catalyze specific biochemical reactions, including RNA splicing in gene expression, similar to the action of protein enzymes | transcript | yes | yes | no
rRNA | RNA, ribosomal - non-protein coding genes that encode ribosomal RNAs (rRNAs) (SO:0001637) | gene | yes | no | no
rRNA | RNA, ribosomal - non-protein coding genes that encode ribosomal RNAs (rRNAs) (SO:0001637) | transcript | yes | no | no
rRNA_pseudogene | Pseudogene: A gene that has homology to known protein-coding genes but contain a frameshift and/or stop codon(s) which disrupts the ORF. Thought to have arisen through duplication followed by loss of function | gene | yes | no | no
rRNA_pseudogene | Pseudogene: A gene that has homology to known protein-coding genes but contain a frameshift and/or stop codon(s) which disrupts the ORF. Thought to have arisen through duplication followed by loss of function | transcript | yes | yes | no
scaRNA | Small Cajal body-specific RNAs are a class of small nucleolar RNAs that specifically localise to the Cajal body, a nuclear organelle involved in the biogenesis of small nuclear ribonucleoproteins | gene | yes | no | no
scaRNA | Small Cajal body-specific RNAs are a class of small nucleolar RNAs that specifically localise to the Cajal body, a nuclear organelle involved in the biogenesis of small nuclear ribonucleoproteins | transcript | yes | no | no
scRNA | RNA, small cytoplasmic - non-protein coding genes that encode small cytoplasmic RNAs (scRNAs) (SO:0001266) | gene | yes | no | no
scRNA | RNA, small cytoplasmic - non-protein coding genes that encode small cytoplasmic RNAs (scRNAs) (SO:0001266) | transcript | yes | no | no
snoRNA | RNA, small nucleolar - non-protein coding genes that encode small nucleolar RNAs (snoRNAs) containing C/D or H/ACA box domains (SO:0001267) | gene | yes | no | no
snoRNA | RNA, small nucleolar - non-protein coding genes that encode small nucleolar RNAs (snoRNAs) containing C/D or H/ACA box domains (SO:0001267) | transcript | yes | no | no
snRNA | RNA, small nuclear - non-protein coding genes that encode small nuclear RNAs (snRNAs) (SO:0001268) | gene | yes | no | no
snRNA | RNA, small nuclear - non-protein coding genes that encode small nuclear RNAs (snRNAs) (SO:0001268) | transcript | yes | no | no
sRNA | Bacterial small RNAs (sRNA) are small RNAs produced by bacteria; they are 50- to 500-nucleotide non-coding RNA molecules, highly structured and containing several stem-loops | gene | yes | no | no
sRNA | Bacterial small RNAs (sRNA) are small RNAs produced by bacteria; they are 50- to 500-nucleotide non-coding RNA molecules, highly structured and containing several stem-loops | transcript | yes | no | no
TEC | TEC (To be Experimentally Confirmed). This is used for non-spliced EST clusters that have polyA features. This category has been specifically created for the ENCODE project to highlight regions that could indicate the presence of protein coding genes that require experimental validation, either by 5' RACE or RT-PCR to extend the transcripts, or by confirming expression of the putatively-encoded peptide with specific antibodies | gene | yes | no | no
TEC | TEC (To be Experimentally Confirmed). This is used for non-spliced EST clusters that have polyA features. This category has been specifically created for the ENCODE project to highlight regions that could indicate the presence of protein coding genes that require experimental validation, either by 5' RACE or RT-PCR to extend the transcripts, or by confirming expression of the putatively-encoded peptide with specific antibodies | transcript | yes | yes | no
TR_C_gene | TR C gene: Constant chain T cell receptor gene that undergoes somatic recombination before transcription | gene | yes | no | no
TR_C_gene | TR C gene: Constant chain T cell receptor gene that undergoes somatic recombination before transcription | transcript | yes | yes | no
TR_D_gene | TR D gene: Diversity chain T cell receptor gene that undergoes somatic recombination before transcription | gene | yes | no | no
TR_D_gene | TR D gene: Diversity chain T cell receptor gene that undergoes somatic recombination before transcription | transcript | yes | yes | no
TR_J_gene | TR J gene: Joining chain T cell receptor gene that undergoes somatic recombination before transcription | gene | yes | no | no
TR_J_gene | TR J gene: Joining chain T cell receptor gene that undergoes somatic recombination before transcription | transcript | yes | yes | no
TR_J_pseudogene | T cell receptor pseudogene - T cell receptor gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame | gene | yes | no | no
TR_J_pseudogene | T cell receptor pseudogene - T cell receptor gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame | transcript | yes | yes | no
TR_V_gene | TR V gene: Variable chain T cell receptor gene that undergoes somatic recombination before transcription | gene | yes | no | no
TR_V_gene | TR V gene: Variable chain T cell receptor gene that undergoes somatic recombination before transcription | transcript | yes | yes | no
TR_V_pseudogene | T cell receptor pseudogene - T cell receptor gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame | gene | yes | no | no
TR_V_pseudogene | T cell receptor pseudogene - T cell receptor gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame | transcript | yes | yes | no
transcribed_processed_pseudogene | Transcribed pseudogene: Pseudogene where protein homology or genomic structure indicates a pseudogene, but the presence of locus-specific transcripts indicates expression. These can be classified into 'Processed', 'Unprocessed' and 'Unitary' | gene | yes | no | no
transcribed_processed_pseudogene | Transcribed pseudogene: Pseudogene where protein homology or genomic structure indicates a pseudogene, but the presence of locus-specific transcripts indicates expression. These can be classified into 'Processed', 'Unprocessed' and 'Unitary' | transcript | yes | no | no
transcribed_unitary_pseudogene | Transcribed pseudogene: Pseudogene where protein homology or genomic structure indicates a pseudogene, but the presence of locus-specific transcripts indicates expression. These can be classified into 'Processed', 'Unprocessed' and 'Unitary' | gene | yes | no | no
transcribed_unitary_pseudogene | Transcribed pseudogene: Pseudogene where protein homology or genomic structure indicates a pseudogene, but the presence of locus-specific transcripts indicates expression. These can be classified into 'Processed', 'Unprocessed' and 'Unitary' | transcript | yes | no | no
transcribed_unprocessed_pseudogene | Transcribed pseudogene: Pseudogene where protein homology or genomic structure indicates a pseudogene, but the presence of locus-specific transcripts indicates expression. These can be classified into 'Processed', 'Unprocessed' and 'Unitary' | gene | yes | no | no
transcribed_unprocessed_pseudogene | Transcribed pseudogene: Pseudogene where protein homology or genomic structure indicates a pseudogene, but the presence of locus-specific transcripts indicates expression. These can be classified into 'Processed', 'Unprocessed' and 'Unitary' | transcript | yes | no | no
translated_processed_pseudogene | Translated pseudogene: Pseudogenes that have mass spec data suggesting that they are also translated. These can be classified into 'Processed', 'Unprocessed' | gene | yes | no | no
translated_processed_pseudogene | Translated pseudogene: Pseudogenes that have mass spec data suggesting that they are also translated. These can be classified into 'Processed', 'Unprocessed' | transcript | yes | yes | no
translated_unprocessed_pseudogene | Translated pseudogene: Pseudogenes that have mass spec data suggesting that they are also translated. These can be classified into 'Processed', 'Unprocessed' | gene | yes | no | no
translated_unprocessed_pseudogene | Translated pseudogene: Pseudogenes that have mass spec data suggesting that they are also translated. These can be classified into 'Processed', 'Unprocessed' | transcript | yes | yes | no
tRNA | tRNA. transfer RNA | gene | yes | no | no
unitary_pseudogene | Unitary pseudogene: A species specific unprocessed pseudogene without a parent gene, as it has an active orthologue in another species | gene | yes | no | no
unitary_pseudogene | Unitary pseudogene: A species specific unprocessed pseudogene without a parent gene, as it has an active orthologue in another species | transcript | yes | yes | no
unknown | entries where the locus type is currently unknown | gene | yes | no | no
unprocessed_pseudogene | Unprocessed pseudogene: Pseudogene that can contain introns since produced by gene duplication | gene | yes | no | no
unprocessed_pseudogene | Unprocessed pseudogene: Pseudogene that can contain introns since produced by gene duplication | transcript | yes | yes | no
vaultRNA | vaultRNA. Short non coding RNA genes that form part of the vault ribonucleoprotein complex | gene | yes | no | no
vaultRNA | vaultRNA. Short non coding RNA genes that form part of the vault ribonucleoprotein complex | transcript | yes | no | no


<br>

**Output:** This script downloads and saves the following data:  
- Human Ensembl Gene Set ➞ [`Homo_sapiens.GRCh38.99.gtf`](ftp://ftp.ensembl.org/pub/release-99/gtf/homo_sapiens/Homo_sapiens.GRCh38.99.gtf.gz)
- Human Ensembl-UniProt Identifiers ➞ [`Homo_sapiens.GRCh38.98.uniprot.tsv`](https://www.dropbox.com/s/cesjvqz1b8c7ami/Homo_sapiens.GRCh38.98.uniprot.tsv?dl=1) 
- Human Ensembl-Entrez Identifiers ➞ [`Homo_sapiens.GRCh38.98.entrez.tsv`](https://www.dropbox.com/s/5kstw70py0azvws/Homo_sapiens.GRCh38.98.entrez.tsv?dl=1) 
- Human Gene Identifiers ➞ [`Homo_sapiens.gene_info`](https://www.dropbox.com/s/vazlmzxydgv6xzz/Homo_sapiens.gene_info?dl=1), [`hgnc_complete_set.txt`](ftp://ftp.ebi.ac.uk/pub/databases/genenames/new/tsv/hgnc_complete_set.txt)  
- Human Protein Identifiers ➞ [`promapping.txt`](https://www.dropbox.com/s/x7wdimv6ph6bl8k/promapping.txt?dl=1)  
- UniProt Identifiers ➞ [`uniprot_identifier_mapping.tab`](https://www.dropbox.com/s/3wbd3wq35328a2v/uniprot_identifier_mapping.tab?dl=1)

_All Merged Data Sets:_ [`Merged_Human_Ensembl_Entrez_HGNC_Uniprot_Identifiers.txt`](https://www.dropbox.com/s/fiek6h5rowi7dh0/Merged_Human_Ensembl_Entrez_HGNC_Uniprot_Identifiers.txt?dl=1)

***

<br>

***
**HGNC Data** 

_Human Gene Set Data_ - `hgnc_complete_set.txt`

In [ ]:
url = 'ftp://ftp.ebi.ac.uk/pub/databases/genenames/new/tsv/hgnc_complete_set.txt'
data_downloader(url, unprocessed_data_location)

In [ ]:
hgnc = pandas.read_csv(unprocessed_data_location + 'hgnc_complete_set.txt',
                       header = 0,
                       delimiter = '\t',
                       low_memory=False)

_Preprocess Data_  
Data file needs to be lightly cleaned before it can be merged with other data. This light cleaning includes renaming columns, replacing `NaN` with `None`, updating data types (i.e. making all columns type `str`), and unnesting `|` delimited data. The final step is to update the gene_type variable such that each of the variable values is re-grouped to be protein-coding, other or ncRNA.

In [ ]:
# remove all rows thave don't have 'approved' status
hgnc = hgnc.loc[hgnc['status'].apply(lambda x: x == 'Approved')]

# drop uneeded columns
hgnc = hgnc[['hgnc_id', 'entrez_id', 'ensembl_gene_id', 'uniprot_ids', 'symbol', 'locus_group',
             'alias_symbol', 'name', 'location', 'alias_name']]

# rename columns
hgnc.rename(columns={'uniprot_ids': 'uniprot_id', 'location': 'map_location'}, inplace=True)

# strip 'HGNC' off of the identifiers
hgnc['hgnc_id'].replace('.*\:','', inplace=True, regex=True)

# combine certain columns into single column
hgnc['primary_symbol'] = hgnc['symbol']
hgnc['symbol'] = hgnc['symbol'] + '|' + hgnc['alias_symbol']
hgnc['name'] = hgnc['name'] + '|' + hgnc['alias_name']
hgnc['synonyms'] = hgnc['alias_symbol'] + '|' + hgnc['alias_name']

# replace NaN with 'None'
hgnc.fillna('None', inplace=True)

# make data columns of type string
hgnc['entrez_id'] = hgnc['entrez_id'].apply(lambda x: str(int(x)) if x != 'None' else 'None')

# explode nested data
explode_df_hgnc = explode(hgnc.copy(), ['ensembl_gene_id', 'uniprot_id', 'symbol', 'name'], '|')

# update cell values
explode_df_hgnc['gene_type_update'] = explode_df_hgnc['locus_group']
explode_df_hgnc['gene_type_update'].replace('protein-coding gene', 'protein-coding', inplace=True, regex=True)
explode_df_hgnc['gene_type_update'].replace('pseudogene', 'other', inplace=True, regex=True)
explode_df_hgnc['gene_type_update'].replace('non-coding RNA', 'ncRNA', inplace=True, regex=True)
explode_df_hgnc['gene_type_update'].replace('phenotype', 'other', inplace=True, regex=True)

# remove original gene type column
explode_df_hgnc.drop(['locus_group', 'alias_symbol', 'alias_name'], axis=1, inplace=True)

# remove duplicates
explode_df_hgnc.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
explode_df_hgnc.head(n=3)
explode_df_hgnc.head(n=3)

<br>

***
***
**Ensembl Data**

_Human Gene Set Data_ - `Homo_sapiens.GRCh38.99.gtf.gz`

In [ ]:
url = 'ftp://ftp.ensembl.org/pub/release-99/gtf/homo_sapiens/Homo_sapiens.GRCh38.99.gtf.gz'
data_downloader(url, unprocessed_data_location)


In [ ]:
ensembl_geneset = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.GRCh38.99.gtf',
                                  header = None,
                                  delimiter = '\t',
                                  skiprows = 5,
                                  low_memory=False)

_Preprocess Data_  
Data file needs to be reformatted in order for it to be able to be merged with the other gene, RNA, and protein identifier data. To do this, we iterate over each row of the data and extract the fields shown below in `column_names`, making each of these extracted fields their own column. The final step is to update the gene_type variable such that each of the variable values is re-grouped to be `protein-coding`, `other` or `ncRNA`.

In [ ]:
# iterate over the nested column and unravel it
column_names = ['gene_id', 'gene_version', 'transcript_id', 'transcript_version', 'exon_number', 'gene_name',
                'gene_source', 'gene_biotype', 'transcript_name', 'transcript_source', 'transcript_biotype',
                'exon_id', 'exon_version', 'tag', 'transcript_support_level']

cleaned_column = []

for idx, row in tqdm(ensembl_geneset.iterrows(), total=ensembl_geneset.shape[0]):
    row_results, col_res = [], row[8].split(';')

    for col in column_names:
        match = [x.replace(col, '').strip().strip('"') for x in col_res if col in x]
        row_results.append(match[0].replace(col, '') if len(match) > 0 else 'None')

    cleaned_column += [row_results]

# add columns back to data frame
ensembl_geneset['ensembl_gene_id'] = [x[0] for x in cleaned_column]
ensembl_geneset['transcript_stable_id'] = [x[2] for x in cleaned_column]
ensembl_geneset['symbol'] = [x[5] for x in cleaned_column]
ensembl_geneset['primary_symbol'] = [x[5] for x in cleaned_column]
ensembl_geneset['gene_type'] = [x[7] for x in cleaned_column]
ensembl_geneset['transcript_type'] = [x[10] for x in cleaned_column]
ensembl_geneset['transcript_name'] = [x[8] for x in cleaned_column]

# replace NaN with 'None'
ensembl_geneset.fillna('None', inplace=True)

# remove uneeded columns
ensembl_geneset.drop(list(range(9)), axis=1, inplace=True)

# remove duplicates
ensembl_geneset.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
ensembl_geneset.head(n=3)

<br>

**Ensembl Annotation Data**

_Ensembl-UniProt_ - `Homo_sapiens.GRCh38.98.uniprot.tsv`  
Once the main ensembl gene set has been read in, the next step is to read in the `ensembl-uniprot` mapping file. These files are vital for successfully merging the ensembl identifiers with the uniprot data set.

In [ ]:
url_uniprot = 'ftp://ftp.ensembl.org/pub/release-99/tsv/homo_sapiens/Homo_sapiens.GRCh38.99.uniprot.tsv.gz'
data_downloader(url_uniprot, unprocessed_data_location)

In [ ]:
ensembl_uniprot = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.GRCh38.99.uniprot.tsv',
                           header = 0,
                           delimiter = '\t',
                           low_memory=False)

# rename columns
ensembl_uniprot.rename(columns={'xref': 'uniprot_id', 'gene_stable_id': 'ensembl_gene_id'}, inplace=True)

# replace "-"
ensembl_uniprot.replace('-', 'None', inplace=True)

# replace NaN with 'None'
ensembl_uniprot.fillna('None', inplace=True)

# remove uneeded columns
ensembl_uniprot.drop(['db_name', 'info_type', 'source_identity', 'xref_identity', 'linkage_type'], axis=1, inplace=True)

# remove duplicates
ensembl_uniprot.drop_duplicates(subset=None, keep='first', inplace=True)

<br>

_Ensembl-Entrez_ - `Homo_sapiens.GRCh38.98.entrez.tsv`  
Once the main ensembl gene set has been read in, the next step is to read in the `ensembl-entrez` mapping file. These files are vital for successfully merging the ensembl identifiers with the entrez data set.

In [ ]:
url_entrez = 'ftp://ftp.ensembl.org/pub/release-99/tsv/homo_sapiens/Homo_sapiens.GRCh38.99.entrez.tsv.gz'
data_downloader(url_entrex, unprocessed_data_location)

In [ ]:
# read in ensembl-entrez data
ensembl_entrez = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.GRCh38.99.entrez.tsv',
                                 header = 0,
                                 delimiter = '\t',
                                 low_memory=False)

# rename columns
ensembl_entrez.rename(columns={'xref': 'entrez_id', 'gene_stable_id': 'ensembl_gene_id'}, inplace=True)

# remove all rows that are not dbname "EntrezGene"
ensembl_entrez = ensembl_entrez.loc[ensembl_entrez['db_name'].apply(lambda x: x == 'EntrezGene')]

# replace "-"
ensembl_entrez.replace('-', 'None', inplace=True)

# replace NaN with 'None'
ensembl_entrez.fillna('None', inplace=True)

# remove uneeded columns
ensembl_entrez.drop(['db_name', 'info_type', 'source_identity', 'xref_identity', 'linkage_type'], axis=1, inplace=True)

# remove duplicates
ensembl_entrez.drop_duplicates(subset=None, keep='first', inplace=True)

<br>

_Merge Annotation Data_ - `ensembl_uniprot` + `ensembl_entrez`

In [ ]:
ensembl_annot = pandas.merge(ensembl_uniprot,
                             ensembl_entrez,
                             left_on=['ensembl_gene_id', 'transcript_stable_id', 'protein_stable_id'],
                             right_on=['ensembl_gene_id', 'transcript_stable_id', 'protein_stable_id'],
                             how='outer')

# replace NaN with 'None'
ensembl_annot.fillna('None', inplace=True)

# preview data
ensembl_annot.head(n=3)

<br>

_Merge Ensembl Annotation and Gene Set Data_ - `ensembl_geneset` + `ensembl_annot`

In [ ]:
ensembl = pandas.merge(ensembl_geneset,
                       ensembl_annot,
                       left_on = ['ensembl_gene_id', 'transcript_stable_id'],
                       right_on = ['ensembl_gene_id', 'transcript_stable_id'],
                       how='outer')

# replace NaN with 'None'
ensembl.fillna('None', inplace=True)
ensembl.replace('NA','None', inplace=True, regex=False)

# preview data
ensembl.head(n=3)

<br>

_Clean Merged Data_  
The final step once the gene set and annotation data have been merged is to update the `gene_type` and `transcript_type` variables such that each of the variable values is re-grouped to be `protein-coding`, `other` or `ncRNA`.

In [ ]:
# update cell values for transcript and gene type
ensembl['gene_type_update'] = ensembl['gene_type']
ensembl['transcript_type_update'] = ensembl['transcript_type']

# protein-coding values
for val in ['protein_coding', 'polymorphic', 'IG_C_gene', 'IG_C_pseudogene', 'IG_D_gene', 'IG_J_gene',
            'IG_J_pseudogene', 'IG_V_gene', 'IG_V_pseudogene', 'TEC', 'TR_C_gene', 'TR_D_gene', 'nonsense_mediated_decay',
            'TR_J_gene', 'TR_J_pseudogene', 'TR_V_gene', 'TR_V_pseudogene', 'translated_processed_pseudogene',
            'translated_unprocessed_pseudogene', 'IG_pseudogene', 'unitary_pseudogene', 'rRNA_pseudogene', 'ribozyme',
            'pseudogene', 'retained_intron', 'polymorphic_pseudogene', 'processed_pseudogene', 'processed_transcript', 'unprocessed_pseudogene']:
    
    if val == 'protein_coding':
        ensembl['gene_type_update'].replace(val, 'protein-coding', inplace=True, regex=False)
        ensembl['transcript_type_update'].replace(val, 'protein-coding', inplace=True, regex=False)
    else:
        ensembl['gene_type_update'].replace(val, 'other', inplace=True, regex=False)
        ensembl['transcript_type_update'].replace(val, 'protein-coding', inplace=True, regex=False)

# noncoding values
for val in ['miRNA', 'vaultRNA', 'lnRNA', 'sRNA', 'scRNA', 'scaRNA', 'snRNA', 'snoRNA', 'lncRNA']:
    ensembl['gene_type_update'].replace(val, 'ncRNA', inplace=True, regex=False)
    ensembl['transcript_type_update'].replace(val, 'ncRNA', inplace=True, regex=False)

# other values
for val in ['rRNA', 'misc_RNA', 'mt_rRNA', 'mt_tRNA', 'Mt_rRNA', 'Mt_tRNA', 'Mt_other','transcribed_processed_pseudogene', 'non_stop_decay',
            'transcribed_unprocessed_pseudogene', 'transcribed_processed_protein-coding', 'transcribed_unitary_pseudogene',
            'transcribed_protein-coding', 'transcribed_unprocessed_protein-coding', 'transcribed_unprotein-coding']:
    
    ensembl['gene_type_update'].replace(val, 'other', inplace=True, regex=False)
    ensembl['transcript_type_update'].replace(val, 'other', inplace=True, regex=False)
        
# remove uneeded columns
ensembl.drop(['gene_type', 'transcript_type'], axis=1, inplace=True)

# remove duplicates
ensembl.drop_duplicates(subset=None, keep='first', inplace=True)
        
# preview data
ensembl.head(n=3)

<br>

***
**UniProt Data**   
_Human Gene Set Data_ - `uniprot_identifier_mapping.tab`

In [ ]:
url = 'https://www.uniprot.org/uniprot/?query=&fil=organism%3A%22Homo%20sapiens%20(Human)%20%5B9606%5D%22&columns=id%2Cdatabase(GeneID)%2Cdatabase(Ensembl)%2Cdatabase(HGNC)%2Cgenes(PREFERRED)%2Cgenes(ALTERNATIVE)&format=tab'
data_downloader(url, unprocessed_data_location, 'uniprot_identifier_mapping.tab')

In [ ]:
uniprot = pandas.read_csv(unprocessed_data_location + 'uniprot_identifier_mapping.tab',
                          header = 0,
                          delimiter = '\t')

_Preprocess Data_  
Data file needs to be lightly cleaned before it can be merged with other data. This light cleaning includes renaming columns, replacing `NaN` with `None`, and unnesting `;` and `" "` delimited data.

In [ ]:
# replace NaN with 'None'
uniprot.fillna('None', inplace=True)

# rename columns
uniprot.rename(columns={'Entry': 'uniprot_id',
                        'Cross-reference (GeneID)': 'entrez_id',
                        'Ensembl transcript': 'transcript_stable_id',
                        'Cross-reference (HGNC)': 'hgnc_id',
                        'Gene names  (synonym )': 'synonyms',
                        'Gene names  (primary )' :'symbol'}, inplace=True)

# combine symbols and synonyms into single column
uniprot['primary_symbol'] = uniprot['symbol']
uniprot['symbol'] = uniprot['symbol'] + ' ' + uniprot['synonyms']

# explode nested data
explode_df_uniprot = explode(uniprot.copy(), ['transcript_stable_id', 'entrez_id', 'hgnc_id'], ';')
explode_df_uniprot = explode(explode_df_uniprot.copy(), ['symbol'], ' ')

# strip out uniprot names
explode_df_uniprot['transcript_stable_id'].replace('\s.*','', inplace=True, regex=True)

# remove duplicates
explode_df_uniprot.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
explode_df_uniprot.head(n=3)

<br>

***
***
**NCBI Data**   
_Human Gene Set Data_ - `Homo_sapiens.gene_info`

In [ ]:
url = 'ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz'
data_downloader(url, unprocessed_data_location)

In [ ]:
ncbi_gene = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.gene_info',
                            header = 0,
                            delimiter = '\t')

_Preprocess Data_  
Data file needs to be lightly cleaned before it can be merged with other data. This light cleaning includes renaming columns, replacing `NaN` with `None`, updating data types (i.e. making all columns type `str`), and unnesting `|` delimited data. Then, the `gene_type` variable is cleaned such that each of the variable's values are re-grouped to be `protein-coding`, `other` or `ncRNA`.

In [ ]:
# remove all rows that are not human
ncbi_gene = ncbi_gene.loc[ncbi_gene['#tax_id'].apply(lambda x: x == 9606)]

# replace "-" with "None"
ncbi_gene.replace('-','None', inplace=True)

# rename columns before merging
ncbi_gene.rename(columns={'GeneID': 'entrez_id', 'Symbol': 'symbol', 'Synonyms': 'synonyms'}, inplace=True)

# combine symbol columns into single column
ncbi_gene['primary_symbol'] = ncbi_gene['symbol']
ncbi_gene['symbol'] = ncbi_gene['Symbol_from_nomenclature_authority'] + '|' + ncbi_gene['symbol'] + '|' + ncbi_gene['synonyms']
ncbi_gene['name'] = ncbi_gene['Full_name_from_nomenclature_authority'] + '|' + ncbi_gene['description']

# explode nested data
explode_df_ncbi_gene = explode(ncbi_gene.copy(), ['symbol', 'name'], '|')

# make sure that merge columns are of same type
explode_df_ncbi_gene['entrez_id'] = explode_df_ncbi_gene['entrez_id'].astype(str)

# update cell values
explode_df_ncbi_gene['gene_type_update'] = explode_df_ncbi_gene['type_of_gene']
explode_df_ncbi_gene['gene_type_update'].replace('pseudo', 'other', inplace=True, regex=True)
explode_df_ncbi_gene['gene_type_update'].replace('biological-region', 'other', inplace=True, regex=True)
explode_df_ncbi_gene['gene_type_update'].replace('tRNA', 'other', inplace=True, regex=True)
explode_df_ncbi_gene['gene_type_update'].replace('rRNA', 'other', inplace=True, regex=True)
explode_df_ncbi_gene['gene_type_update'].replace('scRNA', 'ncRNA', inplace=True, regex=True)
explode_df_ncbi_gene['gene_type_update'].replace('snRNA', 'ncRNA', inplace=True, regex=True)
explode_df_ncbi_gene['gene_type_update'].replace('snoRNA', 'ncRNA', inplace=True, regex=True)

# remove uneeded columns
explode_df_ncbi_gene.drop(['type_of_gene', 'dbXrefs', 'description', 'Nomenclature_status', 'Modification_date',
                           'LocusTag', '#tax_id', 'Full_name_from_nomenclature_authority', 'Feature_type',
                           'Symbol_from_nomenclature_authority'], axis=1, inplace=True)

# remove duplicates
explode_df_ncbi_gene.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
explode_df_ncbi_gene.head(n=3)

<br>

***
**Protein Ontology Identifier Mapping Data**   
_Protein Ontology Identifier Data_ - `promapping.txt`

In [ ]:
url = 'https://proconsortium.org/download/current/promapping.txt'
data_downloader(url, unprocessed_data_location)

In [ ]:
pro_mapping = pandas.read_csv(unprocessed_data_location + 'promapping.txt',
                              header = None,
                              names = ['pro_id', 'Entry', 'pro_mapping'],
                              delimiter = '\t')

_Preprocess Data_  
Basic filtering to to include `Protein Ontology` mappings to `Uniprot` identifiers and cleaning to update formatting of accession values in order to remove `UniProtKB:` is performed.

In [ ]:
# remove rows without 'UniProtKB'
pro_mapping = pro_mapping.loc[pro_mapping['Entry'].apply(lambda x: x.startswith('UniProtKB:'))] 

# remove identifier type, which appears before ':'
pro_mapping['Entry'].replace('(^\w*\:)','', inplace=True, regex=True)

# rename columns before merging
pro_mapping.rename(columns={'Entry': 'uniprot_id'}, inplace=True)

# remove uneeded columns
pro_mapping.drop(['pro_mapping'], axis=1, inplace=True)

# remove duplicates
pro_mapping.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
pro_mapping.head(n=3)

<br>

***
***
**Merge Processed Data:** `hgnc` + `ensembl`

In [ ]:
# merge uniprot and ncbi data
ensembl_hgnc_merged_data = pandas.merge(ensembl,
                                        explode_df_hgnc,
                                        left_on=['ensembl_gene_id',
                                                 'entrez_id',
                                                 'uniprot_id',
                                                 'gene_type_update',
                                                 'symbol',
                                                 'primary_symbol'],
                                        right_on=['ensembl_gene_id',
                                                  'entrez_id',
                                                  'uniprot_id',
                                                  'gene_type_update',
                                                  'symbol',
                                                  'primary_symbol'],
                                        how='outer')

# replace NaN with 'None'
ensembl_hgnc_merged_data.fillna('None', inplace=True)

# remove duplicates
ensembl_hgnc_merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
ensembl_hgnc_merged_data.head(n=3)

<br>

***
***
**Merge Processed Data:** `ensembl_hgnc_merged_data` + `explode_df_uniprot`

In [ ]:
# merge uniprot and ncbi data
ensembl_hgnc_uniprot_merged_data = pandas.merge(ensembl_hgnc_merged_data,
                                                explode_df_uniprot,
                                                left_on=['entrez_id',
                                                         'hgnc_id',
                                                         'uniprot_id',
                                                         'transcript_stable_id',
                                                         'symbol',
                                                         'primary_symbol',
                                                         'synonyms'],
                                                right_on=['entrez_id',
                                                          'hgnc_id',
                                                          'uniprot_id',
                                                          'transcript_stable_id',
                                                          'symbol',
                                                          'primary_symbol',
                                                          'synonyms'],
                                                how='outer')

# replace NaN with 'None'
ensembl_hgnc_uniprot_merged_data.fillna('None', inplace=True)

# remove duplicates
ensembl_hgnc_uniprot_merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
ensembl_hgnc_uniprot_merged_data.head(n=3)

<br>

***
**Merge Processed Data:** `ensembl_hgnc_uniprot_merged_data` + `Homo_sapiens.gene_info`

In [ ]:
ensembl_hgnc_uniprot_ncbi_merged_data = pandas.merge(ensembl_hgnc_uniprot_merged_data,
                                                     explode_df_ncbi_gene,
                                                     left_on=['entrez_id',
                                                              'gene_type_update',
                                                              'symbol',
                                                              'primary_symbol',
                                                              'synonyms',
                                                              'name',
                                                              'map_location'],
                                                     right_on=['entrez_id',
                                                               'gene_type_update',
                                                               'symbol',
                                                               'primary_symbol',
                                                               'synonyms',
                                                               'name',
                                                               'map_location'],
                                                     how='outer')

# replace NaN with 'None'
ensembl_hgnc_uniprot_ncbi_merged_data.fillna('None', inplace=True)

# remove duplicates
ensembl_hgnc_uniprot_ncbi_merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
ensembl_hgnc_uniprot_ncbi_merged_data.head(n=3)

<br>

***

**Merge Processed Data:** `ensembl_hgnc_uniprot_ncbi_merged_data` + `promapping.txt`  

In [ ]:
merged_data = pandas.merge(ensembl_hgnc_uniprot_ncbi_merged_data,
                           pro_mapping,
                           left_on='uniprot_id',
                           right_on='uniprot_id',
                           how='outer')

# replace NaN with 'None'
merged_data.fillna('None', inplace=True)

# remove duplicates
merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
merged_data.head(n=3)

<br>

_Fix Symbol Formatting_  
some genes are formatted similarly to dates (e.g. `DEC1`), which can be erroneously re-formatted during input as a date value (i.e. `1-DEC`). In order for the data to be successfully merged with other data sources, all date-formatted genes need to be resolved.

In [ ]:
clean_dates = []

for x in tqdm(list(merged_data['symbol'])):
    if '-' in x and len(x.split('-')[0]) < 3 and len(x.split('-')[1]) == 3:
        clean_dates.append(x.split('-')[1].upper() + x.split('-')[0])
    else:
        clean_dates.append(x)

# add cleaned date var back to data set
merged_data['symbol'] = clean_dates

# remove duplicates
merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

<br>

_Write Merged Data_

In [ ]:
# remove duplicates
merged_data_clean = merged_data.drop_duplicates(subset=None, keep='first')

# write data
merged_data_clean.to_csv(processed_data_location + 'Merged_Human_Ensembl_Entrez_HGNC_Uniprot_Identifiers.txt',
                         header = True,
                         sep = '\t',
                         index = False)
    
# preview data
merged_data_clean.head(n=3)

<br>

**Create a Master Mapping Dictionary**  
Although the above steps result in a `pandas.Dataframe` of the merged identifiers, there is still work needed in order to be able to obtain a complete mapping between the identifiers. For example, if you were to search for entrez identifier `entrez_259234` you would find the following mappings: `entrez_259234-ENSG00000233316`, `entrez_259234-DSCR10`. If you only had `ENSG00000233316`, from the current data you would be unable to obtain the gene symbol without first mapping to the Entrez gene identifier. 

To solve this problem, we build a master dictionary where the keys are `ensembl_gene_id`, `transcript_stable_id`, `symbol`, `protein_stable_id`, `uniprot_id`, `entrez_id`, `hgnc_id`, and `pro_id` identifiers and values are the list of identifiers that match to each identifier. It's important to note that there are several labeling identifiers (i.e. `name`, `primary_symbol`, `chromosome`, `map_location`, `Other_designations`, `synonyms`, `transcript_name`, `gene_type_update`, and `trasnscript_type_update`), which will only be mapped when clustered against one of the primary identifier types (i.e. the keys described above).

_Note_. The next chunk takes approximately ~80 minutes to run.

In [ ]:
# get all permutations of identifiers (e.g. ['entrez_id', 'ensembl_gene_id'])
identifiers = ['ensembl_gene_id', 'transcript_stable_id', 'symbol', 'primary_symbol',
               'protein_stable_id', 'uniprot_id', 'entrez_id', 'gene_type_update',
               'transcript_type_update', 'hgnc_id', 'name', 'map_location', 'synonyms',
               'chromosome', 'Other_designations', 'pro_id', 'transcript_name']

identifier_list = list(itertools.permutations(identifiers, 2))

# create master dictionary of all identifiers
master_dict = {}

for ids in tqdm(identifier_list):
    if ids[0] not in ['primary_symbol', 'gene_type_update', 'transcript_type_update', 'name',
                      'map_location', 'synonyms', 'chromosome', 'Other_designations', 'transcript_name']:
        
        print(ids)
        
        maps = {k: [ids[1] + '_' + x for x in set(g[ids[1]].tolist()) if x != 'None'] for k,g in merged_data.groupby(ids[0])}
        
        for key in tqdm(maps.keys()):
            if ids[0] + '_' + key in master_dict.keys():
                master_dict[ids[0] + '_' + key] += maps[key]
            else:
                master_dict[ids[0] + '_' + key] = maps[key]

# save a copy of the dictionary
json.dump(master_dict, open(processed_data_location + 'Merged_gene_rna_protein_identifiers.json', 'w'))


_Write Dictionary Locally_  
Convert dictionary to a pairwise list of identifiers and include a third column that contains the gene or transcript type mapping. 

In [ ]:
# set globals
gene_type_var = 'gene_type_update'
transcript_type_var = 'transcript_type_update'

# loop over dict and create master dicitonary
master_map_list = []

for ident in tqdm(master_dict.keys()):    
    mapped_ints = master_dict[ident].copy()
    
    # get gene and transcript types
    gene_types = [x.split('_')[-1] for x in mapped_ints if x.startswith(gene_type_var)]
    trans_types = [x.split('_')[-1] for x in mapped_ints if x.startswith(transcript_type_var)]
    mapped_ints = [x for x in mapped_ints if not x.startswith(gene_type_var) and not x.startswith(transcript_type_var)]
    
    # clean gene and transcript types mapped to multiple types
    updates = []
    for types in [gene_types if len(gene_types) > 0 else ['None'], trans_types if len(trans_types) > 0 else ['None']]:
        if len(set(types)) == 1:
            updates.append(list(set(types))[0])
        elif 'protein-coding' in set(types):
            updates.append('protein-coding')
        else:
            updates.append('not protein-coding')
    
    gene_type = updates[0]
    trans_type = updates[1]
    
    # iterate over identifiers and add to master list
    for mapped_id in mapped_ints:
        master_map_list.append([ident, mapped_id, gene_type, trans_type])


<br>

***
***

### Ensembl Gene-Entrez Gene <a class="anchor" id="ensemblgene-entrezgene"></a>


**Purpose:** To map Ensembl gene identifiers to Entrez gene identifiers when creating the following edges:   
- gene-gene

**Output:** [`ENSEMBL_GENE_ENTREZ_GENE_MAP.txt`](https://www.dropbox.com/s/crghjh2we5v7pws/ENSEMBL_GENE_ENTREZ_GENE_MAP.txt?dl=1)

In [ ]:
# reformat data and write it out
with open(processed_data_location + 'ENSEMBL_GENE_ENTREZ_GENE_MAP.txt', 'w') as outfile:
    for row in tqdm(master_map_list):
        if 'ensembl_gene_id' in row[0] and 'entrez_id' in row[1]:
            outfile.write(row[0].split('_')[-1] + '\t' + row[1].split('_')[-1] + '\t' + row[2] + '\n')

outfile.close()


_Preview Processed Data_

In [ ]:
egeg_data = pandas.read_csv(processed_data_location + 'ENSEMBL_GENE_ENTREZ_GENE_MAP.txt',
                            header = None,
                            names=['Ensembl_Gene_IDs', 'Entrez_Gene_IDs', 'Gene_Type'],
                            delimiter = '\t')

print('There are {edge_count} ensembl gene-entrez gene edges'.format(edge_count=len(egeg_data)))


In [ ]:
egeg_data.head(n=5)

<br>

***

### Ensembl Transcript-Protein Ontology <a class="anchor" id="ensembltranscript-proteinontology"></a>

**Purpose:** To map Ensembl transcript identifiers to Protein Ontology identifiers when creating the following edges: 
- rna-protein  

**Output:** [`ENSEMBL_TRANSCRIPT_PROTEIN_ONTOLOGY_MAP.txt`](https://www.dropbox.com/s/ckrw11nfyu6a08c/ENSEMBL_TRANSCRIPT_PROTEIN_ONTOLOGY_MAP.txt?dl=1)


In [ ]:
# reformat data and write it out
with open(processed_data_location + 'ENSEMBL_TRANSCRIPT_PROTEIN_ONTOLOGY_MAP.txt', 'w') as outfile:
    for row in tqdm(master_map_list):
        if 'transcript_stable_id' in row[0] and 'pro_id' in row[1]:
            outfile.write(row[0].split('_')[-1] + '\t' + row[1].split('_')[-1].replace(':', '_') + '\t' + row[3] + '\n')

outfile.close()



_Preview Processed Data_

In [ ]:
etpr_data = pandas.read_csv(processed_data_location + 'ENSEMBL_TRANSCRIPT_PROTEIN_ONTOLOGY_MAP.txt',
                            header = None,
                            names=['Ensembl_Transcript_IDs', 'Protein_Ontology_IDs', 'Transcript_Type'],
                            delimiter = '\t',
                            low_memory=False)

print('There are {edge_count} ensembl transcript-protein ontology edges'.format(edge_count=len(etpr_data)))

In [ ]:
etpr_data.head(n=5)

<br>

***
***

### Entrez Gene-Ensembl Transcript <a class="anchor" id="entrezgene-ensembltranscript"></a>

**Purpose:** To map entrez gene identifiers to Ensembl transcript identifiers when creating the following edges: 
- gene-rna 

**Output:** [`ENTREZ_GENE_ENSEMBL_TRANSCRIPT_MAP.txt`](https://www.dropbox.com/s/yqnofd8h90luygu/ENTREZ_GENE_ENSEMBL_TRANSCRIPT_MAP.txt?dl=1)

In [ ]:
# reformat data and write it out
with open(processed_data_location + 'ENTREZ_GENE_ENSEMBL_TRANSCRIPT_MAP.txt', 'w') as outfile:
    for row in tqdm(master_map_list):
        if 'entrez_id' in row[0] and 'transcript_stable_id' in row[1]:
            outfile.write(row[0].split('_')[-1] + '\t' + row[1].split('_')[-1] + '\t' + row[3] + '\t' + row[2] + '\n')

outfile.close()


_Preview Processed Data_

In [ ]:
eet_data = pandas.read_csv(processed_data_location + 'ENTREZ_GENE_ENSEMBL_TRANSCRIPT_MAP.txt',
                            header=None,
                            names=['Entrez_Gene_IDs', 'Ensembl_Transcript_IDs', 'Gene_Type', 'Transcript_Type'],
                            delimiter='\t',
                           low_memory=False)

print('There are {edge_count} entrez gene identifiers-ensembl transcript edges'.format(edge_count=len(eet_data)))

In [ ]:
eet_data.head(n=5)

<br>

***

### Entrez Gene-Protein Ontology <a class="anchor" id="entrezgene-proteinontology"></a>

**Purpose:** To map Protein Ontology identifiers to Ensembl transcript identifiers when creating the following edges:   
- chemical-protein  
- gene-protein

**Output:** [`ENTREZ_GENE_PRO_ONTOLOGY_MAP.txt`](https://www.dropbox.com/s/ufbp5o6zgagriw7/ENTREZ_GENE_PRO_ONTOLOGY_MAP.txt?dl=1)

In [ ]:
# reformat data and write it out
with open(processed_data_location + 'ENTREZ_GENE_PRO_ONTOLOGY_MAP.txt', 'w') as outfile:
    for row in tqdm(master_map_list):
        if 'entrez_id' in row[0] and 'pro_id' in row[1]:
            outfile.write(row[0].split('_')[-1] + '\t' + row[1].split('_')[-1].replace(':', '_') + '\t' + row[2] + '\n')

outfile.close()


_Preview Processed Data_

In [ ]:
egpr_data = pandas.read_csv(processed_data_location + 'ENTREZ_GENE_PRO_ONTOLOGY_MAP.txt',
                            header = None,
                            names=['Gene_IDs', 'Protein_Ontology_IDs', 'Gene_Type'],
                            delimiter = '\t')

print('There are {edge_count} entrez gene-protein ontology edges'.format(edge_count=len(egpr_data)))

In [ ]:
egpr_data.head(n=5)

<br>

***
***

### Gene Symbol-Ensembl Transcript <a class="anchor" id="genesymbol-ensembltranscript"></a>

**Purpose:** To map gene symbols to Ensembl transcript identifiers when creating the following edges: 
- chemical-rna  
- rna-anatomy  
- rna-cell  

**Output:** [`GENE_SYMBOL_ENSEMBL_TRANSCRIPT_MAP.txt`](https://www.dropbox.com/s/5o8yt7eejbf819x/GENE_SYMBOL_ENSEMBL_TRANSCRIPT_MAP.txt?dl=1)

In [ ]:
# reformat data and write it out
with open(processed_data_location + 'GENE_SYMBOL_ENSEMBL_TRANSCRIPT_MAP.txt', 'w') as outfile:
    for row in tqdm(master_map_list):
        if 'symbol' in row[0] and 'transcript_stable_id' in row[1]:
            outfile.write(row[0].split('_')[-1] + '\t' + row[1].split('_')[-1] + '\t' + row[2] + '\t' + row[2] + '\n')

outfile.close()


_Preview Processed Data_

In [ ]:
set_data = pandas.read_csv(processed_data_location + 'GENE_SYMBOL_ENSEMBL_TRANSCRIPT_MAP.txt',
                            header = None,
                            names=['Gene_Symbols', 'Ensembl_Transcript_IDs', 'Gene_Type', 'Transcript_Type'],
                            delimiter = '\t')

print('There are {edge_count} gene symbol-ensembl transcript edges'.format(edge_count=len(set_data)))

In [ ]:
set_data.head(n=5)

<BR>

***

### STRING-Protein Ontology <a class="anchor" id="string-proteinontology"></a>

**Purpose:** To map STRING identifiers to Protein Ontology identifiers when creating the following edges:   
- protein-protein  

**Output:** [`STRING_PRO_ONTOLOGY_MAP.txt`](https://www.dropbox.com/s/mekh5lr3bxp7gvu/STRING_PRO_ONTOLOGY_MAP.txt?dl=1)

In [ ]:
# reformat data and write it out
with open(processed_data_location + 'STRING_PRO_ONTOLOGY_MAP.txt', 'w') as outfile:
    for row in tqdm(master_map_list):
        if 'protein_stable_id' in row[0] and 'pro_id' in row[1]:
            outfile.write('9606.' + row[0].split('_')[-1] + '\t' + row[1].split('_')[-1].replace(':', '_') + '\n')

outfile.close()



_Preview Processed Data_

In [ ]:
stpr_data = pandas.read_csv(processed_data_location + 'STRING_PRO_ONTOLOGY_MAP.txt',
                            header = None,
                            names=['STRING_IDs', 'Protein_Ontology_IDs'],
                            delimiter = '\t')

print('There are {edge_count} string-protein ontology edges'.format(edge_count=len(stpr_data)))

In [ ]:
stpr_data.head(n=5)

<br>

***
***

### Uniprot Accession-Protein Ontology <a class="anchor" id="uniprotaccession-proteinontology"></a>

**Purpose:** To map Uniprot accession identifiers to Protein Ontology identifiers when creating the following edges:  
- protein-gobp  
- protein-gomf  
- protein-gocc  
- protein-complex  
- protein-cofactor  
- protein-catalyst 
- protein-reaction  
- protein-pathway

**Output:** [`UNIPROT_ACCESSION_PRO_ONTOLOGY_MAP.txt`](https://www.dropbox.com/s/txp8tqdipzwus9p/UNIPROT_ACCESSION_PRO_ONTOLOGY_MAP.txt?dl=1)

In [ ]:
# reformat data and write it out
with open(processed_data_location + 'UNIPROT_ACCESSION_PRO_ONTOLOGY_MAP.txt', 'w') as outfile:
    for row in tqdm(master_map_list):
        if 'uniprot_id' in row[0] and 'pro_id' in row[1]:
            outfile.write(row[0].split('_')[-1] + '\t' + row[1].split('_')[-1].replace(':', '_') + '\n')

outfile.close()


_Preview Processed Data_

In [ ]:
uapr_data = pandas.read_csv(processed_data_location + 'UNIPROT_ACCESSION_PRO_ONTOLOGY_MAP.txt',
                            header = None,
                            names=['Uniprot_Accession_IDs', 'Protein_Ontology_IDs'],
                            delimiter = '\t')

print('There are {edge_count} uniprot accession-protein ontology edges'.format(edge_count=len(uapr_data)))

In [ ]:
uapr_data.head(n=5)

<br><br>

***
***
### Other Identifier Mapping <a class="anchor" id="other-identifier-mapping"></a>
***
* [ChEBI Identifiers](#mesh-chebi)  
* [Human Protein Atlas Tissue and Cell Types](#hpa-uberon) 
* [Human Disease and Phenotype Identifiers](#disease-identifiers) 

***
***

### ChEBI-MeSH Identifiers <a class="anchor" id="mesh-chebi"></a>

**Data Source Wiki Page:** [mapping-mesh-to-chebi](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#mapping-mesh-identifiers-to-chebi-identifiers)  

**Purpose:** Map MeSH identifiers to ChEBI identifiers when creating the following edges:  
- chemical-gene  
- chemical-disease

**Dependencies:** This script assumes that the [`ncbo_rest_api.py`](https://github.com/callahantiff/PheKnowLator/blob/development/scripts/python/ncbo_rest_api.py) script was run and the data generated from this file was written to `./resources/processed_data/temp`. 

**Output:** [`MESH_CHEBI_MAP.txt`](https://www.dropbox.com/s/5nr87v5h6x8oc1b/MESH_CHEBI_MAP.txt?dl=1)


In [ ]:
with open(processed_data_location + 'MESH_CHEBI_MAP.txt', 'w') as out:
    for filename in tqdm(glob.glob(processed_data_location + 'temp/*.txt')):
        for row in list(filter(None, open(filename, 'r').read().split('\n'))):
            mesh = '_'.join(row.split('\t')[0].split('/')[-2:])
            chebi = row.split('\t')[1].split('/')[-1]
            out.write(mesh + '\t' + chebi + '\n')

out.close()

_Preview Processed Data_

In [ ]:
mc_data = pandas.read_csv(processed_data_location + 'MESH_CHEBI_MAP.txt',
                          delimiter = '\t',
                          header=None,
                          names=['MeSH_IDs', 'ChEBI_IDs'])

print('There are {edge_count} MeSH-ChEBI edges'.format(edge_count=len(mc_data)))

In [ ]:
mc_data.head(n=5)

<br>

***

### Disease and Phenotype Identifiers <a class="anchor" id="disease-identifiers"></a>

**Data Source Wiki Page:** [disgenet](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#disgenet)  

**Purpose:** This script downloads the [disease_mappings.tsv](https://www.disgenet.org/static/disgenet_ap1/files/downloads/disease_mappings.tsv.gz) to map UMLS identifiers to Human Disease and Human Phenotype identifiers when creating the following edges:  
- chemical-disease  
- disease-phenotype

**Output:**   
- Human Disease Ontology Mappings ➞ [`DISEASE_DOID_MAP.txt`](https://www.dropbox.com/s/q30ferujl7k574j/DISEASE_DOID_MAP.txt?dl=1)  
- Human Phenotype Ontology Mappings ➞ [`PHENOTYPE_HPO_MAP.txt`](https://www.dropbox.com/s/5ayl0c5qm7r4tdm/PHENOTYPE_HPO_MAP.txt?dl=1)

In [ ]:
url = 'https://www.disgenet.org/static/disgenet_ap1/files/downloads/disease_mappings.tsv.gz'
data_downloader(url, unprocessed_data_location)

In [ ]:
disease_data = pandas.read_csv(unprocessed_data_location + 'disease_mappings.tsv',
                               header = 0,
                               delimiter = '|')

disease_data.head(n=3)

_Build Disease Identifier Dictionary_  
In order to improve efficiency when mapping different disease terminology identifiers to the [Human Disease Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#human-disease-ontology) and [Human Phenotype Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#human-phenotype-ontology), we create a dictionary of disease identifiers.

In [ ]:
# convert to dictionary
disease_dict = {}

for idx, row in tqdm(disease_data.iterrows(), total=disease_data.shape[0]):
    if row['vocabulary'] == 'MSH':
        mesh_finder(disease_data, row['code'], 'MESH:', disease_dict)
    elif row['vocabulary'] == 'OMIM':
        mesh_finder(disease_data, row['code'], 'OMIM:', disease_dict)
    elif row['vocabulary'] == 'ORDO':
        mesh_finder(disease_data, row['code'], 'ORPHA:', disease_dict)
    elif row['diseaseId'] in disease_dict.keys():
        if row['vocabulary'] == 'DO':
            disease_dict[row['diseaseId']].append('DOID_' + row['code']) 
        if row['vocabulary'] == 'HPO':
            disease_dict[row['diseaseId']].append(row['code'].replace('HP:', 'HP_'))
    else:
        if row['vocabulary'] == 'DO':
            disease_dict[row['diseaseId']] = ['DOID_' + row['code']] 
        if row['vocabulary'] == 'HPO':
            disease_dict[row['diseaseId']] = [row['code'].replace('HP:', 'HP_')] 

_Write Mapping Data_

In [ ]:
with open(processed_data_location + 'DISEASE_DOID_MAP.txt', 'w') as outfile1,open(processed_data_location + 'PHENOTYPE_HPO_MAP.txt', 'w') as outfile2:
    for key, value in tqdm(disease_dict.items()):
        for i in value:
            # get diseases
            if i.startswith('DOID_'): 
                outfile1.write(key.split(':')[-1] + '\t' + i + '\n')

            # get phenotypes
            if i.startswith('HP_'): 
                outfile2.write(key.split(':')[-1] + '\t' + i + '\n')

outfile1.close()
outfile2.close()

<br>

_Preview Processed Human Disease Ontology Mappings_

In [ ]:
dis_data = pandas.read_csv(processed_data_location + 'DISEASE_DOID_MAP.txt',
                           header = None,
                           names=['Disease_IDs', 'DOID_IDs'],
                           delimiter = '\t')

print('There are {} disease-DOID edges'.format(len(dis_data)))

In [ ]:
dis_data.head(n=5)

<br>

_Preview Processed Human Phenotype Mappings_

In [ ]:
hp_data = pandas.read_csv(processed_data_location + 'PHENOTYPE_HPO_MAP.txt',
                          header = None,
                          names=['Disease_IDs', 'HP_IDs'],
                          delimiter = '\t')

print('There are {} phenotype-HPO edges'.format(len(hp_data)))

In [ ]:
hp_data.head(n=5)

<br>

***

### Human Protein Atlas/GTEx Tissue/Cells - UBERON + Cell Ontology + Cell Line Ontology <a class="anchor" id="hpa-uberon"></a>

**Data Source Wiki Page:**  
- [human-protein-atlas](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#human-protein-atlas) 
- [genotype-tissue-expression-project](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#the-genotype-tissue-expression-gtex-project)  

<br>

**Purpose:** Downloads a query for cell, tissue, and blood types with overexpressed protein-coding genes in the human proteome ([`proteinatlas_search.tsv`](https://www.proteinatlas.org/api/search_download.php?search=&columns=g,eg,up,pe,rnatsm,rnaclsm,rnacasm,rnabrsm,rnabcsm,rnablsm,scl,t_RNA_adipose_tissue,t_RNA_adrenal_gland,t_RNA_amygdala,t_RNA_appendix,t_RNA_basal_ganglia,t_RNA_bone_marrow,t_RNA_breast,t_RNA_cerebellum,t_RNA_cerebral_cortex,t_RNA_cervix,_uterine,t_RNA_colon,t_RNA_corpus_callosum,t_RNA_ductus_deferens,t_RNA_duodenum,t_RNA_endometrium_1,t_RNA_epididymis,t_RNA_esophagus,t_RNA_fallopian_tube,t_RNA_gallbladder,t_RNA_heart_muscle,t_RNA_hippocampal_formation,t_RNA_hypothalamus,t_RNA_kidney,t_RNA_liver,t_RNA_lung,t_RNA_lymph_node,t_RNA_midbrain,t_RNA_olfactory_region,t_RNA_ovary,t_RNA_pancreas,t_RNA_parathyroid_gland,t_RNA_pituitary_gland,t_RNA_placenta,t_RNA_pons_and_medulla,t_RNA_prostate,t_RNA_rectum,t_RNA_retina,t_RNA_salivary_gland,t_RNA_seminal_vesicle,t_RNA_skeletal_muscle,t_RNA_skin_1,t_RNA_small_intestine,t_RNA_smooth_muscle,t_RNA_spinal_cord,t_RNA_spleen,t_RNA_stomach_1,t_RNA_testis,t_RNA_thalamus,t_RNA_thymus,t_RNA_thyroid_gland,t_RNA_tongue,t_RNA_tonsil,t_RNA_urinary_bladder,t_RNA_vagina,t_RNA_B-cells,t_RNA_dendritic_cells,t_RNA_granulocytes,t_RNA_monocytes,t_RNA_NK-cells,t_RNA_T-cells,t_RNA_total_PBMC,cell_RNA_A-431,cell_RNA_A549,cell_RNA_AF22,cell_RNA_AN3-CA,cell_RNA_ASC_diff,cell_RNA_ASC_TERT1,cell_RNA_BEWO,cell_RNA_BJ,cell_RNA_BJ_hTERT+,cell_RNA_BJ_hTERT+_SV40_Large_T+,cell_RNA_BJ_hTERT+_SV40_Large_T+_RasG12V,cell_RNA_CACO-2,cell_RNA_CAPAN-2,cell_RNA_Daudi,cell_RNA_EFO-21,cell_RNA_fHDF/TERT166,cell_RNA_HaCaT,cell_RNA_HAP1,cell_RNA_HBEC3-KT,cell_RNA_HBF_TERT88,cell_RNA_HDLM-2,cell_RNA_HEK_293,cell_RNA_HEL,cell_RNA_HeLa,cell_RNA_Hep_G2,cell_RNA_HHSteC,cell_RNA_HL-60,cell_RNA_HMC-1,cell_RNA_HSkMC,cell_RNA_hTCEpi,cell_RNA_hTEC/SVTERT24-B,cell_RNA_hTERT-HME1,cell_RNA_HUVEC_TERT2,cell_RNA_K-562,cell_RNA_Karpas-707,cell_RNA_LHCN-M2,cell_RNA_MCF7,cell_RNA_MOLT-4,cell_RNA_NB-4,cell_RNA_NTERA-2,cell_RNA_PC-3,cell_RNA_REH,cell_RNA_RH-30,cell_RNA_RPMI-8226,cell_RNA_RPTEC_TERT1,cell_RNA_RT4,cell_RNA_SCLC-21H,cell_RNA_SH-SY5Y,cell_RNA_SiHa,cell_RNA_SK-BR-3,cell_RNA_SK-MEL-30,cell_RNA_T-47d,cell_RNA_THP-1,cell_RNA_TIME,cell_RNA_U-138_MG,cell_RNA_U-2_OS,cell_RNA_U-2197,cell_RNA_U-251_MG,cell_RNA_U-266/70,cell_RNA_U-266/84,cell_RNA_U-698,cell_RNA_U-87_MG,cell_RNA_U-937,cell_RNA_WM-115,blood_RNA_basophil,blood_RNA_classical_monocyte,blood_RNA_eosinophil,blood_RNA_gdT-cell,blood_RNA_intermediate_monocyte,blood_RNA_MAIT_T-cell,blood_RNA_memory_B-cell,blood_RNA_memory_CD4_T-cell,blood_RNA_memory_CD8_T-cell,blood_RNA_myeloid_DC,blood_RNA_naive_B-cell,blood_RNA_naive_CD4_T-cell,blood_RNA_naive_CD8_T-cell,blood_RNA_neutrophil,blood_RNA_NK-cell,blood_RNA_non-classical_monocyte,blood_RNA_plasmacytoid_DC,blood_RNA_T-reg,blood_RNA_total_PBMC,brain_RNA_amygdala,brain_RNA_basal_ganglia,brain_RNA_cerebellum,brain_RNA_cerebral_cortex,brain_RNA_hippocampal_formation,brain_RNA_hypothalamus,brain_RNA_midbrain,brain_RNA_olfactory_region,brain_RNA_pons_and_medulla,brain_RNA_thalamus&format=tsv)) and median gene-level TPM by tissue for all genes that are not protein-coding ([`GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct`](https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz)) in order to create mappings between cell and tissue type strings to the Uber-Anatomy, Cell Ontology, and Cell Line Ontology concepts (see [human-protein-atlas](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#human-protein-atlas) for details on the mapping process). The mappings are then used to create the following edge types:  
- rna-cell line  
- rna-tissue type   
- protein-cell line  
- protein-tissue type  

<br>

**Output:**  
- All HPA tissue and cell type strings ➞ [`HPA_tissues.txt`](https://www.dropbox.com/s/m0spn8h1l8kxb61/HPA_tissues.txt?dl=1)  
- Mapping HPA strings to ontology concepts (documentation) ➞ [`zooma_tissue_cell_mapping_04JAN2020.xlsx`](https://www.dropbox.com/s/u7elnc056zxypc6/HPA_GTEX_RNA_GENE_PROTEIN_EDGES.txt?dl=1)  
- Final HPA-ontology mappings ➞ [`HPA_GTEx_TISSUE_CELL_MAP.txt`](https://www.dropbox.com/s/snzdwv1cvs0v9pp/HPA_GTEx_TISSUE_CELL_MAP.txt?dl=1)
- HPA Edges ➞ [`HPA_GTEX_RNA_GENE_PROTEIN_EDGES.txt`](https://www.dropbox.com/s/u7elnc056zxypc6/HPA_GTEX_RNA_GENE_PROTEIN_EDGES.txt?dl=1)

**Human Protein Atlas**

_Download Data_

In [ ]:
url = 'https://www.proteinatlas.org/api/search_download.php?search=&columns=g,eg,up,pe,rnatsm,rnaclsm,rnacasm,rnabrsm,rnabcsm,rnablsm,scl,t_RNA_adipose_tissue,t_RNA_adrenal_gland,t_RNA_amygdala,t_RNA_appendix,t_RNA_basal_ganglia,t_RNA_bone_marrow,t_RNA_breast,t_RNA_cerebellum,t_RNA_cerebral_cortex,t_RNA_cervix,_uterine,t_RNA_colon,t_RNA_corpus_callosum,t_RNA_ductus_deferens,t_RNA_duodenum,t_RNA_endometrium_1,t_RNA_epididymis,t_RNA_esophagus,t_RNA_fallopian_tube,t_RNA_gallbladder,t_RNA_heart_muscle,t_RNA_hippocampal_formation,t_RNA_hypothalamus,t_RNA_kidney,t_RNA_liver,t_RNA_lung,t_RNA_lymph_node,t_RNA_midbrain,t_RNA_olfactory_region,t_RNA_ovary,t_RNA_pancreas,t_RNA_parathyroid_gland,t_RNA_pituitary_gland,t_RNA_placenta,t_RNA_pons_and_medulla,t_RNA_prostate,t_RNA_rectum,t_RNA_retina,t_RNA_salivary_gland,t_RNA_seminal_vesicle,t_RNA_skeletal_muscle,t_RNA_skin_1,t_RNA_small_intestine,t_RNA_smooth_muscle,t_RNA_spinal_cord,t_RNA_spleen,t_RNA_stomach_1,t_RNA_testis,t_RNA_thalamus,t_RNA_thymus,t_RNA_thyroid_gland,t_RNA_tongue,t_RNA_tonsil,t_RNA_urinary_bladder,t_RNA_vagina,t_RNA_B-cells,t_RNA_dendritic_cells,t_RNA_granulocytes,t_RNA_monocytes,t_RNA_NK-cells,t_RNA_T-cells,t_RNA_total_PBMC,cell_RNA_A-431,cell_RNA_A549,cell_RNA_AF22,cell_RNA_AN3-CA,cell_RNA_ASC_diff,cell_RNA_ASC_TERT1,cell_RNA_BEWO,cell_RNA_BJ,cell_RNA_BJ_hTERT+,cell_RNA_BJ_hTERT+_SV40_Large_T+,cell_RNA_BJ_hTERT+_SV40_Large_T+_RasG12V,cell_RNA_CACO-2,cell_RNA_CAPAN-2,cell_RNA_Daudi,cell_RNA_EFO-21,cell_RNA_fHDF/TERT166,cell_RNA_HaCaT,cell_RNA_HAP1,cell_RNA_HBEC3-KT,cell_RNA_HBF_TERT88,cell_RNA_HDLM-2,cell_RNA_HEK_293,cell_RNA_HEL,cell_RNA_HeLa,cell_RNA_Hep_G2,cell_RNA_HHSteC,cell_RNA_HL-60,cell_RNA_HMC-1,cell_RNA_HSkMC,cell_RNA_hTCEpi,cell_RNA_hTEC/SVTERT24-B,cell_RNA_hTERT-HME1,cell_RNA_HUVEC_TERT2,cell_RNA_K-562,cell_RNA_Karpas-707,cell_RNA_LHCN-M2,cell_RNA_MCF7,cell_RNA_MOLT-4,cell_RNA_NB-4,cell_RNA_NTERA-2,cell_RNA_PC-3,cell_RNA_REH,cell_RNA_RH-30,cell_RNA_RPMI-8226,cell_RNA_RPTEC_TERT1,cell_RNA_RT4,cell_RNA_SCLC-21H,cell_RNA_SH-SY5Y,cell_RNA_SiHa,cell_RNA_SK-BR-3,cell_RNA_SK-MEL-30,cell_RNA_T-47d,cell_RNA_THP-1,cell_RNA_TIME,cell_RNA_U-138_MG,cell_RNA_U-2_OS,cell_RNA_U-2197,cell_RNA_U-251_MG,cell_RNA_U-266/70,cell_RNA_U-266/84,cell_RNA_U-698,cell_RNA_U-87_MG,cell_RNA_U-937,cell_RNA_WM-115,blood_RNA_basophil,blood_RNA_classical_monocyte,blood_RNA_eosinophil,blood_RNA_gdT-cell,blood_RNA_intermediate_monocyte,blood_RNA_MAIT_T-cell,blood_RNA_memory_B-cell,blood_RNA_memory_CD4_T-cell,blood_RNA_memory_CD8_T-cell,blood_RNA_myeloid_DC,blood_RNA_naive_B-cell,blood_RNA_naive_CD4_T-cell,blood_RNA_naive_CD8_T-cell,blood_RNA_neutrophil,blood_RNA_NK-cell,blood_RNA_non-classical_monocyte,blood_RNA_plasmacytoid_DC,blood_RNA_T-reg,blood_RNA_total_PBMC,brain_RNA_amygdala,brain_RNA_basal_ganglia,brain_RNA_cerebellum,brain_RNA_cerebral_cortex,brain_RNA_hippocampal_formation,brain_RNA_hypothalamus,brain_RNA_midbrain,brain_RNA_olfactory_region,brain_RNA_pons_and_medulla,brain_RNA_thalamus&format=tsv'
data_downloader(url, unprocessed_data_location, 'proteinatlas_search.tsv.gz')

_Load Data_

In [ ]:
hpa = pandas.read_csv(unprocessed_data_location + 'proteinatlas_search.tsv',
                      header = 0,
                      delimiter = '\t')

# replace NaN with 'None'
hpa.fillna('None', inplace=True)

_Identify HPA Terms Needing Mapping_  
To expedite the mapping process, all HPA tissues, cells, cell lines, and fluid types are extracted from the HPA data columns.

In [ ]:
# retrieve terms to map
terms_to_map = list(hpa.columns)

# write results
with open(unprocessed_data_location + 'HPA_tissues.txt', 'w') as outfile:
    for x in tqdm(terms_to_map):
        if x.endswith('[NX]'):
            term = x.split('RNA - ')[-1].split(' [NX]')[:-1][0]
            outfile.write(term + '\n')

outfile.close()

<br>

**Genotype-Tissue Expression Project**

_Download Data_

In [ ]:
url='https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz'
data_downloader(url, unprocessed_data_location)

_Load Data_

In [ ]:
gtex = pandas.read_csv(unprocessed_data_location + 'GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct',
                      header = 0,
                      skiprows=2,
                      delimiter = '\t')

# replace NaN with 'None'
gtex.fillna('None', inplace=True)

# # set index of ensembl ids as column 'Ensembl'
# gtex['Ensembl'] = gtex.index

# remove identifier type, which appears after '.'
gtex['Name'].replace('(\..*)','', inplace=True, regex=True)

<br>

**Get Mapping Data**   
Import the tissues, cells, cell lines, and fluids that we externally mapped from HPA and GTEx data to [UBERON](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#uber-anatomy-ontology), the [Cell Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#cell-ontology), and the [Cell Line Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#cell-line-ontology).

In [ ]:
mapping_data = pandas.read_excel(open(unprocessed_data_location + 'zooma_tissue_cell_mapping_04JAN2020.xlsx', 'rb'),
                                 sheet_name='Concept_Mapping - 04JAN2020',
                                 header=0)

# convert NaN to None
mapping_data.fillna('None', inplace=True)

# preview data
mapping_data.head(n=3)

<br>

_Write HPA and GTEx Mapping Data_  
The HPA and GTEx mapping data is written locally so that it can be used by the `PheKnowLator` algorithm when creating the knowledge graph edge lists. 

In [ ]:
with open(processed_data_location + 'HPA_GTEx_TISSUE_CELL_MAP.txt', 'w') as outfile:
    for idx, row in tqdm(mapping_data.iterrows(), total=mapping_data.shape[0]):
        if row['UBERON ID'] != 'None':
            outfile.write(str(row['ORIGINAL TERM']).strip() + '\t' + str(row['UBERON ID']).strip() + '\n')
        if row['CL ID'] != 'None':
            outfile.write(str(row['ORIGINAL TERM']).strip() + '\t' + str(row['CL ID']).strip() + '\n')
        if row['CLO ID'] != 'None':
            outfile.write(str(row['ORIGINAL TERM']).strip() + '\t' + str(row['CLO ID']).strip() + '\n')

outfile.close()

_Preview Processed Data_

In [ ]:
mapping_data = pandas.read_csv(processed_data_location + 'HPA_GTEx_TISSUE_CELL_MAP.txt',
                               header = None,
                               names=['TISSUE_CELL_TERM', 'ONTOLOGY_IDs'],
                               delimiter = '\t')

In [ ]:
mapping_data.head(n=3)

<br>

**Create Edge Data Set**

_Human Protein Atlas_  
The `HPA` data looped over and reformatted such all all tissue, cell, cell lines, and fluid types are stored as a nested list. As shown in the code chunk, you will see that the anatomy type is specified as an item in the list according to its type. This is done in order to make mapping more efficient while building the knowledge graph edge list.

In [ ]:
hpa_results = []

for idx, row in tqdm(hpa.iterrows(), total=hpa.shape[0]):
    if row['RNA tissue specific NX'] != 'None':
        for x in row['RNA tissue specific NX'].split(';'):
            hpa_results += [[row['Ensembl'], row['Gene'], row['Uniprot'], row['Evidence'], 'anatomy', x.split(':')[0]]]

    if row['RNA cell line specific NX'] != 'None':
        for x in row['RNA cell line specific NX'].split(';'):
            hpa_results += [[row['Ensembl'], row['Gene'], row['Uniprot'], row['Evidence'], 'cell line', x.split(':')[0]]]

    if row['RNA brain regional specific NX'] != 'None':
        for x in row['RNA brain regional specific NX'].split(';'):
            hpa_results += [[row['Ensembl'], row['Gene'], row['Uniprot'], row['Evidence'], 'anatomy', x.split(':')[0]]]

    if row['RNA blood cell specific NX'] != 'None':
        for x in row['RNA blood cell specific NX'].split(';'):
            hpa_results += [[row['Ensembl'], row['Gene'], row['Uniprot'], row['Evidence'], 'anatomy', x.split(':')[0]]]

    if row['RNA blood lineage specific NX'] != 'None':
        for x in row['RNA blood lineage specific NX'].split(';'):
            hpa_results += [[row['Ensembl'], row['Gene'], row['Uniprot'], row['Evidence'], 'anatomy', x.split(':')[0]]]

<br>

_Genotype-Tissue Expression Project_  
The `GTEx` edge data is created by first filtering out all _protein-coding_ genes that appear in the `HPA` cell transcriptome data set. Once filter so that we are only left noncoding genes, we perform an additional filtering step to only add genes and their corresponding tissue, cell, or fluid, if the median expression is `>= 1.0`. The `GTEx` is formatted such all all tissue, cell, and fluid types occur as their own column and all unique genes occur as a row, thus the expression filtering step is performed while also reformatting the file. The genes and tissues/cells/fluids that meet criteria are stored as a nested list.

In [ ]:
# get a list of hpa protein-coding genes
hpa_genes = list(hpa['Ensembl'].drop_duplicates(keep='first', inplace=False))

# remove rows that contain protein coding genes
gtex = gtex.loc[gtex['Name'].apply(lambda x: x not in hpa_genes)]

# loop over data and re-organize - only keep results with tpm >= 1 and if gene symbol is not a protein-coding gene
gtex_results = []

for idx, row in tqdm(gtex.iterrows(), total=gtex.shape[0]):    
    for col in list(gtex.columns)[2:]:
        if row[col] >= 1.0:           
            gtex_results += [[row['Name'], row['Description'], 'None', 'Evidence at transcript level', 'cell line' if 'Cells' in col else 'anatomy', col]]

_Write Results_

In [ ]:
with open(processed_data_location + 'HPA_GTEX_RNA_GENE_PROTEIN_EDGES.txt', 'w') as outfile:
    for res in tqdm(hpa_results + gtex_results):
        outfile.write(str(res[0]) + '\t' + str(res[1]) + '\t' + str(res[2]) + '\t' + str(res[3]) + '\t' + str(res[4]) + '\t' + str(res[5]) + '\n')

outfile.close()

_Preview Processed Data_

In [ ]:
hpa_edges = pandas.read_csv(processed_data_location + 'HPA_GTEX_RNA_GENE_PROTEIN_EDGES.txt',
                           header = None,
                           names=['Ensembl_IDs', 'Gene_Symbols', 'Uniport_IDs', 'Evidence', 'Anatomy_Type', 'Anatomy'],
                           low_memory=False,
                           delimiter = '\t')

print('There are {edge_count} edges'.format(edge_count=len(hpa_edges)))

In [ ]:
hpa_edges.head(n=5)

<br>

***
***
### CREATE EDGE DATASETS  <a class="anchor" id="create-edge-datasets"></a>
***
***

### Ontologies  <a class="anchor" id="ontologies"></a>
***
- [Protein Ontology](#protein-ontology)  
- [Relations Ontology](#relations-ontology)  

***
***

### Protein Ontology <a class="anchor" id="protein-ontology"></a>

**Data Source Wiki Page:** [protein-ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#human-phenotype-ontology)  

**Purpose:** This script downloads the [pr.owl](http://purl.obolibrary.org/obo/pr.owl) file from [ProConsortium.org](https://proconsortium.org/) in order to create a version of the ontology that contains only human proteins. This is achieved by performing forward and reverse breadth first search over all proteins which are `owl:subClassOf` [Homo sapiens protein](https://proconsortium.org/app/entry/PR%3A000029067/).

<br>

**Output:**  
- Human Protein Ontology ➞ [`human_pro.owl`](https://www.dropbox.com/s/jw8jksgnqbcz9sm/human_pro.owl?dl=1)
- Classified Human Protein Ontology (Hermit) ➞ [`human_pro_closed.owl`](https://www.dropbox.com/s/6ux85agl95ja3wx/human_pro_closed.owl?dl=1)


In [ ]:
url = 'http://purl.obolibrary.org/obo/pr.owl'
data_downloader(url, unprocessed_data_location)

In [ ]:
# read in ontology as graph (the ontology is large so this takes ~60 minutes) - 11,757,623 edges on 12/18/2019
graph = Graph()
graph.parse(unprocessed_data_location + 'pr.owl')

print('There are {} edges in the ontology'.format(len(graph)))

_Convert Ontology to Directed MulitGraph_  
In order to create a version of the ontology which includes all relevant human edges, we need to first convert the KG to a [directed multigraph](https://networkx.github.io/documentation/stable/reference/classes/multidigraph.html).

In [ ]:
# convert RDF graph to multidigraph (the ontology is large so this takes ~45 minutes)
networkx_mdg = rdflib_to_networkx_multidigraph(graph)

_Identify Human Proteins_   
A list of human proteins is obtained by querying the ontology to return all ontology classes `only_in_taxon some Homo sapiens`. To expedite the query time, the following SPARQL query is run from the [ProConsortium](https://proconsortium.org/pro_sparql.shtml) SPARQL endpoint: 

```SPARQL
PREFIX obo: <http://purl.obolibrary.org/obo/>

SELECT ?PRO_term
FROM <http://purl.obolibrary.org/obo/pr>
WHERE {
       ?PRO_term rdf:type owl:Class .
       ?PRO_term rdfs:subClassOf ?restriction .
       ?restriction owl:onProperty obo:RO_0002160 .
       ?restriction owl:someValuesFrom obo:NCBITaxon_9606 .

       # use this to filter-out things like hgnc ids
       FILTER (regex(?PRO_term,"http://purl.obolibrary.org/obo/*")) .
}

```


In [ ]:
# download data - pro classes only_in_taxon some Homo sapiens (61,064 classes on 12/18/2019)
url = 'http://sparql.proconsortium.org/virtuoso/sparql?query=PREFIX+obo%3A+%3Chttp%3A%2F%2Fpurl.obolibrary.org%2Fobo%2F%3E%0D%0ASELECT+%3FPRO_term%0D%0AFROM+%3Chttp%3A%2F%2Fpurl.obolibrary.org%2Fobo%2Fpr%3E%0D%0AWHERE%0D%0A%7B%0D%0A+++%3FPRO_term+rdf%3Atype+owl%3AClass+.%0D%0A+++%3FPRO_term+rdfs%3AsubClassOf+%3Frestriction+.%0D%0A+++%3Frestriction+owl%3AonProperty+obo%3ARO_0002160+.%0D%0A+++%3Frestriction+owl%3AsomeValuesFrom+obo%3ANCBITaxon_9606+.%0D%0A%0D%0A+++FILTER+%28regex%28%3FPRO_term%2C%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2F*%22%29%29+.%0D%0A%0D%0A%7D%0D%0A&format=text%2Fhtml&debug='
html = requests.get(url, allow_redirects=True).content

# extract data from html table
df_list = pandas.read_html(html)
human_pro_classes = list(df_list[-1]['PRO_term'])

print('There are {protein_count} human classes in the PRO ontology'.format(protein_count=len(human_pro_classes)))

<br>

_Construct Human PRO_   
Now that we have all of the paths from the original graph that are relevant to humans, we can construct a human-only version of the PRotein ontology.

In [ ]:
# create a new graph using bfs paths
human_pro_graph = Graph()
human_networkx_mdg = networkx.MultiDiGraph()

for node in tqdm(human_pro_classes):
    forward = list(networkx.edge_bfs(networkx_mdg, URIRef(node), orientation='original'))
    reverse = list(networkx.edge_bfs(networkx_mdg, URIRef(node), orientation='reverse'))
    
    # add edges from forward and reverse bfs paths
    for path in forward + reverse:
        human_pro_graph.add((path[0], path[2], path[1]))
        human_networkx_mdg.add_edge(path[0], path[1], path[2])

In [ ]:
# verify that the constructed ontology only has 1 component
networkx.number_connected_components(human_networkx_mdg.to_undirected())

In [ ]:
# save filtered ontology
human_pro_graph.serialize(destination=unprocessed_data_location + 'human_pro.owl', format='xml')

<br>

_Classify Ontology_  
To ensure that we have correclty built the new ontology, we run the hermit reasoner over it to ensure that there are no incomplete triples or inconsistent classes. In order to do this, we will call the reasoner using [OWLTools](https://github.com/owlcollab/owltools), which this script assumes has already been downloaded to the `../resources/lib` directory. The following arguments are then called to run the reasoner (from the command line):  

```bash
./resources/lib/owltools ./resources/unprocessed_data/human_pro_filtered.owl --reasoner hermit --run-reasoner --assert-implied -o ./resources/processed_data/human_pro_closed.owl
```

_**Note.** This step takes around 30-45 minutes to run. When run from the command line the reasoner determined that the ontology was consistent and 174 new axioms were inferrred (12/18/2019)._

In [ ]:
# run reasoner -- RUN FROM COMMAND LINE NOT HERE
# subprocess.run(['../../resources/lib/owltools',
#                 '../../resources/unprocessed_data/human_pro_filtered.owl',
#                 '--reasoner hermit',
#                 '--run-reasoner',
#                 '--assert-implied',
#                 '--list-unsatisfiable',
#                 '-o ./resources/processed_data/human_pro_closed.owl'])

<br>

_Examine Cleaned Human PRO_  
Once we have cleaned the ontology we can get counts of components, nodes, edges, and then write the cleaned graph to the `../../resources/processed_data` repository.

In [ ]:
# get count of connected components
pro_human_graph = Graph()
pro_human_graph.parse(processed_data_location + 'human_pro_closed.owl')

# get node and edge count
edge_count = len(human_pro_graph)
node_count = len(set([str(node) for edge in list(human_pro_graph) for node in edge[0::2]]))

print('\n The classified, filtered Human version of PRO contains {node} nodes and {edge} edges\n'.format(node=node_count, edge=edge_count))

<br>

***
***

### Relations Ontology <a class="anchor" id="relations-ontology"></a>

**Data Source Wiki Page:** [RO](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#relation-ontology)  

**Purpose:** This script downloads the [ro.owl](http://purl.obolibrary.org/obo/ro.owl) file from [obofoundry.org](http://www.obofoundry.org/) in order to obtain all `ObjectProperties` and their inverse relations.  

**Output:** 
- Relations and Inverse Relations ➞ [`INVERSE_RELATIONS.txt`](https://www.dropbox.com/s/sd8qlib8f6gqyz4/INVERSE_RELATIONS.txt?dl=1)
- Relations and Labels ➞ [`RELATIONS_LABELS.txt`](https://www.dropbox.com/s/k2hm9p0r8l9ecj3/RELATIONS_LABELS.txt?dl=1)

_Download Ontology_

In [ ]:
# download ontology using subprocess and OWLTOOLS in order to get the ontology and its imported ontologies
subprocess.check_call(['./resources/lib/owltools',
                       'http://purl.obolibrary.org/obo/ro.owl',
                       '--merge-import-closure',
                       '-o',
                       unprocessed_data_location + 'ro_with_imports.owl'])

_Load Ontology to RDFLib Graph_

In [ ]:
ro_graph = Graph()
ro_graph.parse(unprocessed_data_location + 'ro_with_imports.owl')

print('There are {} edges in the ontology (date:{})'.format(len(ro_graph), datetime.datetime.now().strftime('%m/%d/%Y')))




**Identify Relations and Inverse Relations**  
Identify all relations and their inverse relations using the `owl:inverseOf` property. To make it easier to look up the inverse relations, each pair is listed twice, for example:  
- [location of](http://www.ontobee.org/ontology/RO?iri=http://purl.obolibrary.org/obo/RO_0001015) `owl:inverseOf` [located in](http://www.ontobee.org/ontology/RO?iri=http://purl.obolibrary.org/obo/RO_0001025)  
- [located in](http://www.ontobee.org/ontology/RO?iri=http://purl.obolibrary.org/obo/RO_0001025) `owl:inverseOf` [location of](http://www.ontobee.org/ontology/RO?iri=http://purl.obolibrary.org/obo/RO_0001015)

In [ ]:
with open(relations_data_location + 'INVERSE_RELATIONS.txt', 'w') as outfile:
    
    # write column names
    outfile.write('Relation' + '\t' + 'Inverse_Relation' + '\n')

    # find inverse relations
    for s, p, o in tqdm(ro_graph):
        if 'owl#inverseOf' in str(p):
            if 'RO' in str(s) and 'RO' in str(o):
                outfile.write(str(s.split('/')[-1]) + '\t' + str(o.split('/')[-1]) + '\n')
                outfile.write(str(o.split('/')[-1]) + '\t' + str(s.split('/')[-1]) + '\n')

outfile.close()

_Preview Processed Data_

In [ ]:
ro_data = pandas.read_csv(relations_data_location + 'INVERSE_RELATIONS.txt',
                          header = 0,
                          delimiter = '\t')

print('There are {edge_count} RO Relations and Inverse Relations'.format(edge_count=len(ro_data)))

In [ ]:
ro_data.head(n=5)

<br>

**Get Relations Labels**  
Identify all relations and their labels for use when building the knowledge graph.

In [ ]:
results = ro_graph.query(
    """SELECT DISTINCT ?p ?p_label
           WHERE {
              ?p rdf:type owl:ObjectProperty .
              ?p rdfs:label ?p_label . }
           """, initNs={"rdf": 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
                        "rdfs": 'http://www.w3.org/2000/01/rdf-schema#',
                        "owl": 'http://www.w3.org/2002/07/owl#'})    

In [ ]:
# write data to file
with open(relations_data_location + 'RELATIONS_LABELS.txt', 'w') as outfile:
    
    # write column names
    outfile.write('Relation' + '\t' + 'Label' + '\n')

    for p, p_label in list(results):
        outfile.write(str(p).split('/')[-1] + '\t' + str(p_label) + '\n')

_Preview Processed Data_

In [ ]:
ro_data_label = pandas.read_csv(relations_data_location + 'RELATIONS_LABELS.txt',
                                header = 0,
                                delimiter = '\t')

print('There are {edge_count} RO Relations and Labels'.format(edge_count=len(ro_data_label)))

In [ ]:
ro_data_label.head(n=5)

<br><br>

***
***
### Linked Data <a class="anchor" id="linked-data"></a>
***
* [Clinvar Variant-Diseases and Phenotypes](#clinvar-variant)
* [Reactome Chemical-Complex Data](#reactome-chemical-complex)  
* [Reactome Complex-Complex Data](#reactome-complex-complex)  
* [Reactome Protein-Complex Data](#reactome-protein-complex)  
* [Uniprot Protein-Cofactor and Protein-Catalyst](#uniprot-protein-cofactorcatalyst)  

***

### Clinvar Variant-Diseases and Phenotypes <a class="anchor" id="clinvar-variant"></a>

**Data Source Wiki Page:** [Clinvar](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#clinvar)  

**Purpose:** This script downloads the [variant_summary.txt](ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz) file from [CLinVar](https://www.ncbi.nlm.nih.gov/clinvar/) in order to create the following edges:  
- gene-variant  
- variant-disease  
- variant-phenotype  

**Output:** [`CLINVAR_VARIANT_GENE_DISEASE_PHENOTYPE_EDGES.txt`](https://www.dropbox.com/s/1doj3lj46ufgdpd/CLINVAR_VARIANT_GENE_DISEASE_PHENOTYPE_EDGES.txt?dl=1)


In [ ]:
url = 'ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz'
data_downloader(url, unprocessed_data_location)

In [ ]:
clinvar_data = pandas.read_csv(unprocessed_data_location + 'variant_summary.txt',
                               header = 0,
                               delimiter = '\t',
                               low_memory=False)

_Preprocess Data_

In [ ]:
# replace NaN with 'None'
clinvar_data.fillna('None', inplace=True)

# explode nested data
explode_df_clinvar = explode(clinvar_data.copy(), ['PhenotypeIDS'], ';')
explode_df_clinvar = explode(explode_df_clinvar.copy(), ['PhenotypeIDS'], ',')

# edit column formatting
explode_df_clinvar['PhenotypeIDS'].replace('Orphanet:ORPHA','ORPHA:', inplace=True, regex=True)
explode_df_clinvar['PhenotypeIDS'].replace('Human Phenotype Ontology:HP:','HP_', inplace=True, regex=True)

# write data
explode_df_clinvar.to_csv(processed_data_location + 'CLINVAR_VARIANT_GENE_DISEASE_PHENOTYPE_EDGES.txt',
                          header = True,
                          sep='\t',
                          encoding='utf-8',
                          index=False)

_Preview Processed Data_

In [ ]:
print('There are {edge_count} variant edges'.format(edge_count=len(explode_df_clinvar)))

In [ ]:
# preview data
explode_df_clinvar.head(n=5)

<br>

***

### Reactome Chemical-Complex Data <a class="anchor" id="reactome-chemical-complex"></a>

**Data Souurce Wiki Page:** [Reactome](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#reactome-pathway-database)  

**Purpose:** This script downloads the [ComplexParticipantsPubMedIdentifiers_human.txt](https://reactome.org/download/current/ComplexParticipantsPubMedIdentifiers_human.txt) file from [Reactome](https://reactome.orgt) in order to create the following edges:  
- chemical-complex  

**Output:** [`REACTOME_CHEMICAL_COMPLEX.txt`](https://www.dropbox.com/s/qoetjt0vfy6qb3y/REACTOME_CHEMICAL_COMPLEX.txt?dl=1)

In [ ]:
data = open(unprocessed_data_location + 'ComplexParticipantsPubMedIdentifiers_human.txt').readlines()

# reformat data and write it out
with open(processed_data_location + 'REACTOME_CHEMICAL_COMPLEX.txt', 'w') as outfile:
    for line in tqdm(data[1:]):
        row = line.split('\t')
        
        if (row[0].strip().startswith('R-HSA') or row[0].strip().startswith('R-ALL')):
            # find all proteins in a complex
            for x in row[2].split('|'):
                if x.startswith('chebi:'):            
                    outfile.write(x.replace('chebi:', 'CHEBI_') + '\t' + row[0].strip() + '\n')

outfile.close()

_Preview Processed Data_

In [ ]:
cc1_data = pandas.read_csv(processed_data_location + 'REACTOME_CHEMICAL_COMPLEX.txt',
                           header = None,
                           names=['CHEBI_IDs', 'Reactome_IDs'],
                           delimiter = '\t')

print('There are {edge_count} chemical-complex edges'.format(edge_count=len(cc1_data)))

In [ ]:
cc1_data.head(n=5)

<br>

***

### Reactome Complex-Complex Data <a class="anchor" id="reactome-complex-complex"></a>

**Data Source Wiki Page:** [Reactome](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#reactome-pathway-database)  

**Purpose:** This script downloads the [ComplexParticipantsPubMedIdentifiers_human.txt](https://reactome.org/download/current/ComplexParticipantsPubMedIdentifiers_human.txt) file from [Reactome](https://reactome.orgt) in order to create the following edges:  
- complex-complex  

**Output:** [`REACTOME_COMPLEX_COMPLEX.txt`](https://www.dropbox.com/s/sojaq8u3hwfw4jz/REACTOME_COMPLEX_COMPLEX.txt?dl=1)


In [ ]:
data = open(unprocessed_data_location + 'ComplexParticipantsPubMedIdentifiers_human.txt').readlines()

# reformat data and write it out
with open(processed_data_location + 'REACTOME_COMPLEX_COMPLEX.txt', 'w') as outfile:
    for line in tqdm(data[1:]):
        row = line.split('\t')
        
        if row[0].strip().startswith('R-HSA'):
            for x in row[3].split('|'):
                if x.startswith('R-HSA-') or x.startswith('R-ALL-'):            
                    outfile.write(row[0].strip() + '\t' + x.strip() + '\n')

outfile.close()

_Preview Processed Data_

In [ ]:
cc_data = pandas.read_csv(processed_data_location + 'REACTOME_COMPLEX_COMPLEX.txt',
                          header = None,
                          names=['Reactome_Complex_u', 'Reactome_Complex_v'],
                          delimiter = '\t')

print('There are {edge_count} complex-complex edges'.format(edge_count=len(cc_data)))

In [ ]:
cc_data.head(n=5)

<br>

***

### Reactome Complex-Pathway Data <a class="anchor" id="reactome-complex-pathway"></a>

**Data Source Wiki Page:** [Reactome](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#reactome-pathway-database)  

**Purpose:** This script downloads the [Complex_2_Pathway_human.txt](https://reactome.org/download/current/Complex_2_Pathway_human.txt) file from [Reactome](https://reactome.orgt) in order to create the following edges:  
- complex-pathway  

**Output:** [`REACTOME_COMPLEX_PATHWAY.txt`](https://www.dropbox.com/s/my03w16fjw7bt20/REACTOME_COMPLEX_PATHWAY.txt?dl=1)


In [ ]:
url = 'https://reactome.org/download/current/Complex_2_Pathway_human.txt'
data_downloader(url, unprocessed_data_location)

In [ ]:
# process data
data = open(unprocessed_data_location + 'Complex_2_Pathway_human.txt').readlines()

# reformat data and write it out
with open(processed_data_location + 'REACTOME_COMPLEX_PATHWAY.txt', 'w') as outfile:
    for line in tqdm(data[1:]):
        row = line.split('\t')
        if row[0].startswith('R-HSA-'):            
            outfile.write(row[0].strip() + '\t' + row[1].strip() + '\n')

outfile.close()

_Preview Processed Data_

In [ ]:
cp_data = pandas.read_csv(processed_data_location + 'REACTOME_COMPLEX_PATHWAY.txt',
                          header = None,
                          names=['Reactome_Complex', 'Reactome_Pathway'],
                          delimiter = '\t')

print('There are {edge_count} complex-pathway edges'.format(edge_count=len(cp_data)))

In [ ]:
cp_data.head(n=5)

<br>

***
***

### Reactome Protein-Complex Data <a class="anchor" id="reactome-protein-complex"></a>

**Data Source Wiki Page:** [Reactome](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#reactome-pathway-database)  

**Purpose:** This script downloads the [ComplexParticipantsPubMedIdentifiers_human.txt](https://reactome.org/download/current/ComplexParticipantsPubMedIdentifiers_human.txt) file from [Reactome](https://reactome.org) in order to create the following edges:  
- protein-complex

**Output:** [`REACTOME_PROTEIN_COMPLEX.txt`](https://www.dropbox.com/s/7meu0cdz1mrnsz7/REACTOME_PROTEIN_COMPLEX.txt?dl=1)


In [ ]:
url = 'https://reactome.org/download/current/ComplexParticipantsPubMedIdentifiers_human.txt'
data_downloader(url, unprocessed_data_location)

In [ ]:
# process data
data = open(unprocessed_data_location + 'ComplexParticipantsPubMedIdentifiers_human.txt').readlines()

# reformat data and write it out
with open(processed_data_location + 'REACTOME_PROTEIN_COMPLEX.txt', 'w') as outfile:
    for line in tqdm(data):
        row = line.split('\t')
        
        if row[0].strip().startswith('R-HSA'):
            # find all proteins in a complex
            for x in row[2].split('|'):
                if x.startswith('uniprot:'):            
                    outfile.write(x.split(':')[-1].strip() + '\t' + row[0].strip() + '\n')

outfile.close()

_Preview Processed Data_

In [ ]:
pc_data = pandas.read_csv(processed_data_location + 'REACTOME_PROTEIN_COMPLEX.txt',
                       header = None,
                       names=['Uniprot_Protein', 'Reactome_Complex'],
                       delimiter = '\t')

print('There are {edge_count} protein-complex edges'.format(edge_count=len(pc_data)))

In [ ]:
pc_data.head(n=5)

<br>

***

### Uniprot  Protein-Cofactor and Protein-Catalyst <a class="anchor" id="uniprot-protein-cofactorcatalyst"></a>

**Data Source Wiki Page:** [Uniprot](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#uniprot-knowledgebase)  

**Purpose:** This script downloads the [uniprot-cofactor-catalyst.tab](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#uniprot-knowledgebase) file from the [Uniprot Knowledge Base](https://www.uniprot.org) in order to create the following edges:  
- protein-cofactor  
- protein-catalyst  

**Output:**  
- protein-cofactor ➞ [`UNIPROT_PROTEIN_COFACTOR.txt`](https://www.dropbox.com/s/ij9t89botd8nmmj/UNIPROT_PROTEIN_COFACTOR.txt?dl=1)
- protein-catalyst ➞ [`UNIPROT_PROTEIN_CATALYST.txt`](https://www.dropbox.com/s/pvopvs0iq8x3oq2/UNIPROT_PROTEIN_CATALYST.txt?dl=1)


In [ ]:
url = 'https://www.uniprot.org/uniprot/?query=&fil=organism%3A%22Homo%20sapiens%20(Human)%20%5B9606%5D%22&columns=id%2Centry%20name%2Creviewed%2Cdatabase(PRO)%2Cchebi(Cofactor)%2Cchebi(Catalytic%20activity)&format=tab'
data_downloader(url, unprocessed_data_location, 'uniprot-cofactor-catalyst.tab')

In [ ]:
data = open(unprocessed_data_location + 'uniprot-cofactor-catalyst.tab').readlines()

# reformat data and write it out
with open(processed_data_location + 'UNIPROT_PROTEIN_COFACTOR.txt', 'w') as outfile1, open(processed_data_location + 'UNIPROT_PROTEIN_CATALYST.txt', 'w') as outfile2:
    for line in tqdm(data):

        # get cofactors
        if 'CHEBI' in line.split('\t')[4]: 
            for i in line.split('\t')[4].split(';'):
                chebi = i.split('[')[-1].replace(']', '').replace(':', '_')
                outfile1.write('PR_' + line.split('\t')[3].strip(';') + '\t' + chebi + '\n')
        
        # get catalysts
        if 'CHEBI' in line.split('\t')[5]:       
            for i in line.split('\t')[5].split(';'):
                chebi = i.split('[')[-1].replace(']', '').replace(':', '_')
                outfile2.write('PR_' + line.split('\t')[3].strip(';') + '\t' + chebi + '\n')

outfile1.close()
outfile2.close()

<br>

_Preview Cofactor Data_

In [ ]:
pcp1_data = pandas.read_csv(processed_data_location + 'UNIPROT_PROTEIN_COFACTOR.txt',
                            header = None,
                            names=['Protein_Ontology_IDs', 'CHEBI_IDs'],
                            delimiter = '\t')

print('There are {edge_count} protein-cofactor edges'.format(edge_count=len(pcp1_data)))

In [ ]:
pcp1_data.head(n=5)

<br>

_Preview Catalyst Data_

In [ ]:
pcp2_data = pandas.read_csv(processed_data_location + 'UNIPROT_PROTEIN_CATALYST.txt',
                            header = None,
                            names=['Protein_Ontology_IDs', 'CHEBI_IDs'],
                            delimiter = '\t')

print('There are {edge_count} protein-catalyst edges'.format(edge_count=len(pcp2_data)))

In [ ]:
pcp2_data.head(n=5)

<br>

***
***
### INSTANCE METADATA <a class="anchor" id="create-instance-metadata"></a>
***

**Data Source Wiki Page:** [Dependencies](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies/#node-metadata) 

<br>

**Purpose:** The goal of this section is to obtain metadata for each instance data source used in the knowledge graph. To determine which of the edges contains instance data, the [`Master_Edge_List_Dict.json`](https://www.dropbox.com/s/4j0vrwx26dh8hd1/Master_Edge_List_Dict.json?dl=1) file is parsed and saved to a nested dictionary (see example below). 

```python
{
  'complex': {
              'chemical-complex': [[node_1, node_2]...[node_n, node_m]],
              'complex-complex':  [[node_1, node_2]...[node_n, node_m]],
              'complex-pathway':  [[node_1, node_2]...[node_n, node_m]],
              },
     'gene': {
                'chemical-gene':  [[node_1, node_2]...[node_n, node_m]],
                 'gene-disease':  [[node_1, node_2]...[node_n, node_m]],
              }
}
```

<br>

Once this dictionary is created, each major data type (examples shown in the list below) will be processed. For **[`Release V2.0.0`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**, the following are instance data and require the compiling of metadata:
- [Genes](#gene-metadata)
- [RNA](#rna-metadata)
- [Pathways](#pathway-metadata)
- [Complexes](#complex-metadata)
- [Reactions](#reaction-metadata)
- [Variants](#variant-metadata)


<br>

____

**Metadata:** The <u>metadata</u> we will gather includes:  

| **Metadata Type** | **Definition** | **Example Node**  | **Example Node Metadata** | 
| :---: | :---: | :---: | :---: | 
| Label | The primary label or name for the node | `R-HSA-1006173` | "CFH:Host cell surface" |       
| Description | A definition or other useful details about the node | `rs794727058` | This `germline` `single nucleotide variant (allele alteration: C➞T)` located on chromosome `5 (GRCh38: NC_000005.10, start/stop positions (126555930/126555930))` with `pathogenic` clinical significance and a last review date of `2/23/2015` (review status: `criteria provided, single submitter`). |        
| Synonym | Alternative terms used for a node | `81399` | "OR1-1, OR7-21" |           

<br>

The metadata information will be used to create the following edges in the knowledge graph:  
- **Label** ➞ node `rdfs:label`  
- **Description** ➞ node `obo:IAO_0000115` description 
- **Synonyms** ➞ node `oboInOwl:hasExactSynonym` synonym 

<br>

*<b>NOTE.</b> All node metadata datasets are written to the `node_data` directory. The algorithm will look for data in this directory and if it is not there, then no node metadata will be created.*

_____

<br>

### Prepare Metadata Dictionaries
***

**Purpose:** To create the resources needed in order to create metadata dictionaries, which are in turn used to obtain metadata for instance data nodes. This process has the following steps:

**1. [Identify Instance Data Nodes](#identify-instance-data-nodes):** In order to automatically obtain the list of edges that include an instance data source and their corresponding edge lists, the `Master_Edge_List_Dict.json` is read in and processed.  
  - <u>Input Data</u>: [`Master_Edge_List_Dict.json`](./resources/Master_Edge_List_Dict.json)  

<br>

**2. [Generate Metadata Dictionaries](#generate-metadata-dictionaries):** In order to efficiently obtain metadata for the instance data nodes identified in _Step 1_, we first read in the data for each node type (i.e. genes, rna, complexes, pathways, reactions, and variants) and convert it into a dictionary. Then, each metadata dictionary is saved to a `master_metadata_dictionary`, keyed by node type.
  - <u>Input Datasets</u>:  
    - Genes ➞ [`Homo_sapiens.gene_info`](https://www.dropbox.com/s/f2nz5q6g46u0tth/Merged_gene_rna_protein_identifiers.json?dl=1)    
    - RNA ➞ [`Merged_Human_Ensembl_Entrez_HGNC_Uniprot_Identifiers.txt`](https://www.dropbox.com/s/fiek6h5rowi7dh0/Merged_Human_Ensembl_Entrez_HGNC_Uniprot_Identifiers.txt?dl=1) 
    - Complexes ➞ [`reactome2py API`](https://github.com/reactome/reactome2py)  
    - Pathways ➞ [`reactome2py API`](https://github.com/reactome/reactome2py)  
    - Reactions ➞ [`reactome2py API`](https://github.com/reactome/reactome2py)  
    - Variants ➞ [`variant_summary.txt.gz`](ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz)  

<br>

**3. [Write Metadata Files](#write-metadata-files):** The Instance data node dictionary from _Step 1_ and metadata dictionaries from _Step 2_ are used to write `.txt` files for all `edge-type` data included in the instance node dictionary.

<br>

***

### Identify Instance Data Nodes  <a class="anchor" id="identify-instance-data-nodes"></a>

In [ ]:
# read in data files for each edge type
edge_data = json.load(open('./resources/Master_Edge_List_Dict.json', 'r'))
edge_dict = {key:[edge_data[key]['data_type'], edge_data[key]['edge_list']] for key in edge_data.keys()}

**Sort Data**  
For all edges in the `edge_dict()` that include instance data, we create a new dictionary where each edge type is further organized by node from the edge type that references the instance data (e.g. from the `chemical-gene` edge type, the `gene` node references instance data).

In [ ]:
# sort data files
metadata_file_info = {}

for edge in tqdm(edge_dict.keys()): 
    if 'instance' in edge_dict[edge][0]:
        
        # get instance type
        inst_type = edge.split('-')[edge_dict[edge][0].split('-').index('instance')]
        
        # read in data
        if inst_type in metadata_file_info.keys(): 
            metadata_file_info[inst_type][edge] = {}
            metadata_file_info[inst_type][edge]['data'] = edge_dict[edge][1]
            metadata_file_info[inst_type][edge]['instance_idx'] = edge_dict[edge][0]
            
        else:
            metadata_file_info[inst_type] = {}
            metadata_file_info[inst_type][edge] =  {}
            metadata_file_info[inst_type][edge]['data'] =  edge_dict[edge][1]
            metadata_file_info[inst_type][edge]['instance_idx'] = edge_dict[edge][0]


<br>

***

### Generate Metadata Dictionaries  <a class="anchor" id="generate-metadata-dictionaries"></a>
In this step, the goal is to create a metadata dictionary for each node type that does not rely on API data. In this case, only the **Gene**, **RNA**, and **Variant** nodes require data that is not from an API.


<br>

**Genes Metadata Dictionary**

In [ ]:
# entrez gene data
entrez_gene_data = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.gene_info',
                                   header = 0,
                                   delimiter = '\t',
                                   low_memory=False)

# remove all rows that are not human
entrez_gene_data = entrez_gene_data.loc[entrez_gene_data['#tax_id'].apply(lambda x: x == 9606)]

# replace NaN and '-' with 'None'
entrez_gene_data.fillna('None', inplace=True)
entrez_gene_data.replace('-','None', inplace=True, regex=False)

_Create Gene Metadata Dictionary_  
The nested dictionary of gene metadata is created by looping over the merged data described in the prior column. The `keys` of the dictionary are `Entrez gene identifiers` and the `values` are dictionaries for each metadata type: `symbol`, `description`, and `name`.

In [ ]:
# create metadata
genes, label, description, synonym = [], [], [], []

for idx, row in tqdm(entrez_gene_data.iterrows(), total=entrez_gene_data.shape[0]):
    # node 
    if row['GeneID'] != 'None':
        genes.append(row['GeneID'])
    
    # label -- only want metadata if there is a label
    if row['Symbol'] != 'None' or row['Symbol'] != '':       
        label.append(row['Symbol'])

        # description        
        if row['Full_name_from_nomenclature_authority'] != 'None' and row['type_of_gene'] != 'None' and row['chromosome'] != 'None' and row['map_location'] != 'None':
        
            description.append('{desc} is a {gene} gene that is located on chromosome {chrom} (map_location: {map_loc}).'.format(desc=row['Symbol'],
                                                                                                                                 gene=row['type_of_gene'],
                                                                                                                                 chrom=row['chromosome'],
                                                                                                                                 map_loc=row['map_location']))
            
        else:
            description.append('{desc} is a {gene} gene.'.format(desc=row['Symbol'], gene=row['type_of_gene']))
        
        # synonym        
        if row['Synonyms'] != 'None' or row['Other_designations'] != 'None':
            syns = '|'.join(set([x for x in (row['Synonyms'] + row['Other_designations']).split('|') if x != 'None' or x != '']))
            synonym.append(syns)
        else:
            synonym.append('None')
            
    
# combine into new data frame        
gene_metadata_final = pandas.DataFrame(list(zip(genes, label, description, synonym)), columns =['ID', 'Label', 'Description', 'Synonym'])

# make all variables string
gene_metadata_final = gene_metadata_final.astype(str)

# dedup
gene_metadata_final.drop_duplicates(subset='ID', keep='first', inplace=True)

# convert df to dictionary
gene_metadata_final.set_index('ID', inplace=True)
gene_metadata_dict = gene_metadata_final.to_dict('index')


<br>

**RNA Metadata Dictionary**

In [ ]:
rna_gene_data = pandas.read_csv(processed_data_location + 'Merged_Human_Ensembl_Entrez_HGNC_Uniprot_Identifiers.txt',
                                header = 0,
                                delimiter = '\t',
                                low_memory=False)

_Preprocess Data_  
Normal data preprocess and filtering steps are performed in order to prepare the data for the next step, which converts it to a metadata dictionary.

In [ ]:
# remove rows without identifiers
rna_gene_data = rna_gene_data.loc[rna_gene_data['transcript_stable_id'].apply(lambda x: x != 'None')]

# remove unneede columns
rna_gene_data.drop(['ensembl_gene_id', 'symbol', 'protein_stable_id', 'uniprot_id',
                    'entrez_id', 'hgnc_id', 'pro_id'], axis=1, inplace=True)

# remove duplicates
rna_gene_data.drop_duplicates(subset=['transcript_stable_id', 'transcript_name', 'transcript_type_update'], keep='first', inplace=True)

# replace NaN with 'None'
rna_gene_data.fillna('None', inplace=True)

# get transcript_names
trans_name = {k: g['transcript_name'].tolist() for k,g in rna_gene_data.groupby('transcript_stable_id')}

# get trans type
trans_type = {k: g['transcript_type_update'].tolist() for k,g in rna_gene_data.groupby('transcript_stable_id')}

_Create RNA Metadata Dictionary_  
The nested dictionary of rna metadata is created by looping over the human merged gene, RNA, protein identifier data set ([`Merged_Human_Ensembl_Entrez_HGNC_Uniprot_Identifiers.txt`](https://www.dropbox.com/s/fiek6h5rowi7dh0/Merged_Human_Ensembl_Entrez_HGNC_Uniprot_Identifiers.txt?dl=1)). The `keys` of the dictionary are `Ensembl transcript identifiers` and the `values` are dictionaries for each metadata type: `symbol`, `description`, and `name`.

In [ ]:
# create metadata
rna, label, description, synonym = [], [], [], []

for node in tqdm(set(list(rna_gene_data['transcript_stable_id']))):
    # node
    rna.append(node)
    
    # label info
    label_list = [x for x in trans_name[node] if x != 'None']
    labels = label_list[0] if len(label_list) > 0 else 'None'
    label.append(labels)
    
    # rna type info
    types = set(trans_type[node])
    rna_type = ['/'.join(x) if len(types) > 1 else x for x in types][0]

    if rna_type != 'None':
        # description
        description.append('Transcript {desc} is classified as type {typ}'.format(desc=node, typ=rna_type))
    else:
        # description
        description.append('None')

    # synonym
    synonym.append('None')
    
# combine into new data frame
rna_metadata_final = pandas.DataFrame(list(zip(rna, label, description, synonym)),
                                      columns =['ID', 'Label', 'Description', 'Synonym'])

# convert df to dictionary
rna_metadata_final.set_index('ID', inplace=True)
rna_metadata_dict = rna_metadata_final.to_dict('index')


<br>

**Variant Metadata Dictionary**

_Download Data_  
Only run this code block if the `variant_summary.txt` has not already been downloaded.

In [ ]:
url = 'ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz'
data_downloader(url, unprocessed_data_location)

In [ ]:
var_data = pandas.read_csv(unprocessed_data_location + 'variant_summary.txt',
                           header = 0,
                           delimiter = '\t',
                           low_memory=False)

_Preprocess Data_  
Normal data preprocess and filtering steps are performed in order to prepare the data for the next step, which converts it to a metadata dictionary.

In [ ]:
# remove rows without identifiers
var_data = var_data.loc[var_data['Assembly'].apply(lambda x: x == 'GRCh38')]
var_data = var_data.loc[var_data['RS# (dbSNP)'].apply(lambda x: x != -1)]

# de-dup data
var_metadata = var_data[['#AlleleID', 'Type', 'Name', 'ClinicalSignificance', 'RS# (dbSNP)', 'Origin',
                         'ChromosomeAccession', 'Chromosome', 'Start', 'Stop', 'ReferenceAllele',
                         'Assembly', 'AlternateAllele','Cytogenetic', 'ReviewStatus', 'LastEvaluated']] 

# replace NaN with 'None'
var_metadata.fillna('None', inplace=True)

# remove duplicate dbSNP ids by choosing the most recent reviewed variant
var_metadata.sort_values('LastEvaluated', ascending=False, inplace=True)
var_metadata.drop_duplicates(subset='RS# (dbSNP)', keep='first', inplace=True)

_Create Variant Metadata Dictionary_  
The nested dictionary of rna metadata is created by looping over the human [ClinVar Variant](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#clinvar) identifier data set ([`variant_summary.txt`](https://www.dropbox.com/s/g58b0oduv7sj3ja/variant_summary.txt?dl=1)). The `keys` of the dictionary are `dbSNP identifiers` and the `values` are dictionaries for each metadata type: `symbol`, `description`, and `name`.

In [ ]:
# create metadata
variant, label, description = [], [], []

for idx, row in tqdm(var_metadata.iterrows(), total=var_metadata.shape[0]):
    
    # node
    if row['RS# (dbSNP)'] != 'None':
        variant.append('rs' + str(row['RS# (dbSNP)']))
    
    # label -- only want metadata if there is a label
    if row['Name'] != 'None':
        label.append(row['Name'])
    
        # description
        sent = 'This variant is a {Origin} {Type} that results when a {ReferenceAllele} allele is changed to {AlternateAllele} on chromosome {Chromosome} ({ChromosomeAccession}, start:{Start}/stop:{Stop} positions, cytogenetic location:{Cytogenetic}) and has clinical significance {ClinicalSignificance}. This entry is for the {Assembly} and was last reviewed on {LastEvaluated} with review status "{ReviewStatus}".'
        description.append(sent.format(Origin=row['Origin'], Type=row['Type'], ReferenceAllele=row['ReferenceAllele'],
                                       AlternateAllele=row['AlternateAllele'], Chromosome=row['Chromosome'],
                                       ChromosomeAccession=row['ChromosomeAccession'], Start=row['Start'],
                                       Stop=row['Stop'], Cytogenetic=row['Cytogenetic'], ClinicalSignificance=row['ClinicalSignificance'],
                                       Assembly=row['Assembly'], LastEvaluated=row['LastEvaluated'], ReviewStatus=row['ReviewStatus']))

# combine into new data frame
var_metadata_final = pandas.DataFrame(list(zip(variant, label, description)), columns =['ID', 'Label', 'Description'])

# drop duplicates
var_metadata_final.drop_duplicates(subset=None, keep='first', inplace=True)

# make all variables string
var_metadata_final = var_metadata_final.astype(str)

# convert df to dictionary
var_metadata_final.set_index('ID', inplace=True)
var_metadata_dict = var_metadata_final.to_dict('index')                      

<br>

**Create Master Metadata Dictionary**  
To make it easier to navigate the mapping of each instance node in an edge, a master dictionary is created and keyed by node type. This is most useful when both nodes in an edge are instances, but of different data types (e.g. `gene-rna`).


In [ ]:
master_metadata_dictionary = {'gene': gene_metadata_dict, 'rna': rna_metadata_dict, 'variant': var_metadata_dict}

<br>

***

### Write Metadata Files  <a class="anchor" id="write-metadata-files"></a>   
using the `Master Metadata Dictionary` created in the prior step, all of the `edge-type` data is processed and the resulting data written out `.txt` file to the `./resource/node_data` repository.

- [Genes](#gene-metadata)
- [RNA](#rna-metadata)
- [Pathways](#pathway-metadata)
- [Complexes](#complex-metadata)
- [Reactions](#reaction-metadata)
- [Variants](#variant-metadata)

***

### Genes <a class="anchor" id="gene-metadata"></a>

**Data Source Wiki Pages:** [NCBI Gene](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#ncbi-gene) 

**Output:**  
- chemical-gene ➞ [`chemical-gene_GENE_METADATA.txt`](https://www.dropbox.com/s/fvkqnuk5xhs0huh/chemical-gene_GENE_METADATA.txt?dl=1) 
- gene-disease ➞ [`gene-disease_GENE_METADATA.txt`](https://www.dropbox.com/s/o0y21rx3b829q6d/gene-disease_GENE_METADATA.txt?dl=1) 
- gene-gene ➞ [`gene-gene_GENE_METADATA.txt`](https://www.dropbox.com/s/i4gznnct7rzh7pn/gene-gene_GENE_METADATA.txt?dl=1) 
- gene-pathway ➞ [`gene-pathway_GENE_METADATA.txt`](https://www.dropbox.com/s/yncd95vanhkp0ey/gene-pathway_GENE_METADATA.txt?dl=1) 
- gene-phenotype ➞ [`gene-phenotype_GENE_METADATA.txt`](https://www.dropbox.com/s/jghcoc5xzada011/gene-phenotype_GENE_METADATA.txt?dl=1) 
- gene-protein ➞ [`gene-protein_GENE_METADATA.txt`](https://www.dropbox.com/s/6vu961lna08qn08/gene-protein_GENE_METADATA.txt?dl=1) 
- gene-rna ➞ [`gene-rna_GENE_METADATA.txt`](https://www.dropbox.com/s/vs0kirmugdo9zkd/gene-rna_GENE_METADATA.txt?dl=1) 

In [ ]:
node_type = 'gene'

for edge_type in tqdm(metadata_file_info[node_type]):
    print('\nPROCESSING EDGE TYPE: {}'.format(edge_type))

    # gather vars for processing data
    data = metadata_file_info[node_type][edge_type]['data']
    edge_data_type = metadata_file_info[node_type][edge_type]['instance_idx']
    inst_idx = edge_data_type.split('-').index('instance')

    # get list of nodes to map and the dictionary to use
    # when nodes are of the same type (i.e. gene-gene)
    if (edge_type.split('-')[0] == edge_type.split('-')[1]):
        nodes = set([x for y in data for x in y])
        
        if edge_type.split('-')[0] in master_metadata_dictionary.keys():
            metadata_dictionaries = master_metadata_dictionary[edge_type.split('-')[0]]
            results = metadata_dictionary_mapper(nodes, metadata_dictionaries)
        else:
            results = metadata_api_mapper(list(nodes))
            
    # when nodes are both instances, but different types (i.e. gene-rna)
    elif edge_data_type.split('-')[0] == edge_data_type.split('-')[1]:
        data_res = []

        for node in edge_type.split('-'):
            nodes = set([x[int(edge_type.split('-').index(node))] for x in data])
            
            if node in master_metadata_dictionary:
                metadata_dictionaries = master_metadata_dictionary[node]
                data_res.append(metadata_dictionary_mapper(nodes, metadata_dictionaries))
            else:
                data_res.append(metadata_api_mapper(list(nodes)))
    
        # combine data into single df
        results = pandas.concat(data_res, ignore_index=True)
                
    # when only one node is an instance
    else:
        nodes = set([x[int(inst_idx)] for x in data])
        
        if edge_type.split('-')[int(inst_idx)] in master_metadata_dictionary.keys():
            metadata_dictionaries = master_metadata_dictionary[edge_type.split('-')[int(inst_idx)]]
            results = metadata_dictionary_mapper(nodes, metadata_dictionaries)
        else:
            results = metadata_api_mapper(list(nodes))

    # write data
    results.to_csv(node_data_location + edge_type + '_' + node_type.upper() + '_METADATA.txt', header = True, sep = '\t', index = False)


<br>

***
***

### RNA<a class="anchor" id="rna-metadata"></a>

**Data Source Wiki Pages:**  
- [Ensembl](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#clinvar)  
- [NCBI Gene](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#ncbi-gene) 

**Output:**  
- chemical-rna ➞ [`chemical-rna_RNA_METADATA.txt`](https://www.dropbox.com/s/sm0orl0waq5iqhd/chemical-rna_RNA_METADATA.txt?dl=1) 
- rna-anatomy ➞ [`rna-anatomy_RNA_METADATA.txt`](https://www.dropbox.com/s/plkrunhhusx6mez/rna-anatomy_RNA_METADATA.txt?dl=1) 
- rna-cell ➞ [`rna-cell_RNA_METADATA.txt`](https://www.dropbox.com/s/dld0eadxyyzr44y/rna-cell_RNA_METADATA.txt?dl=1) 
- rna-protein ➞ [`rna-protein_RNA_METADATA.txt`](https://www.dropbox.com/s/3g72sb2e685rptn/rna-protein_RNA_METADATA.txt?dl=1) 

In [ ]:
node_type = 'rna'

for edge_type in tqdm(metadata_file_info[node_type]):
    print('\nPROCESSING EDGE TYPE: {}'.format(edge_type))

    # gather vars for processing data
    data = metadata_file_info[node_type][edge_type]['data']
    edge_data_type = metadata_file_info[node_type][edge_type]['instance_idx']
    inst_idx = edge_data_type.split('-').index('instance')

    # get list of nodes to map and the dictionary to use
    # when nodes are of the same type (i.e. gene-gene)
    if (edge_type.split('-')[0] == edge_type.split('-')[1]):
        nodes = set([x for y in data for x in y])
        
        if edge_type.split('-')[0] in master_metadata_dictionary.keys():
            metadata_dictionaries = master_metadata_dictionary[edge_type.split('-')[0]]
            results = metadata_dictionary_mapper(nodes, metadata_dictionaries)
        else:
            results = metadata_api_mapper(list(nodes))
            
    # when nodes are both instances, but different types (i.e. gene-rna)
    elif edge_data_type.split('-')[0] == edge_data_type.split('-')[1]:
        data_res = []

        for node in edge_type.split('-'):
            nodes = set([x[int(edge_type.split('-').index(node))] for x in data])
            
            if node in master_metadata_dictionary:
                metadata_dictionaries = master_metadata_dictionary[node]
                data_res.append(metadata_dictionary_mapper(nodes, metadata_dictionaries))
            else:
                data_res.append(metadata_api_mapper(list(nodes)))
    
        # combine data into single df
        results = pandas.concat(data_res, ignore_index=True)
                
    # when only one node is an instance
    else:
        nodes = set([x[int(inst_idx)] for x in data])
        
        if edge_type.split('-')[int(inst_idx)] in master_metadata_dictionary.keys():
            metadata_dictionaries = master_metadata_dictionary[edge_type.split('-')[int(inst_idx)]]
            results = metadata_dictionary_mapper(nodes, metadata_dictionaries)
        else:
            results = metadata_api_mapper(list(nodes))

    # write data
    results.to_csv(node_data_location + edge_type + '_' + node_type.upper() + '_METADATA.txt', header = True, sep = '\t', index = False)


<br>

***

### Pathways<a class="anchor" id="pathway-metadata"></a>

**Data Source Wiki Page:** [Reactome](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#reactome-pathway-database)  

**Output:**    
- chemical-pathway ➞ [`chemical-pathway_PATHWAY_METADATA.txt`](https://www.dropbox.com/s/2txg2ui4e6y7rnm/chemical-pathway_PATHWAY_METADATA.txt?dl=1)
- gobp-pathway ➞ [`gobp-pathway_PATHWAY_METADATA.txt`](https://www.dropbox.com/s/bq0g1g4ef40vwxj/gobp-pathway_PATHWAY_METADATA.txt?dl=1)
- pathway-gocc ➞ [`pathway-gocc_PATHWAY_METADATA.txt`](https://www.dropbox.com/s/6fzkzjxj08u6jfi/pathway-gocc_PATHWAY_METADATA.txt?dl=1)
- pathway-gomf ➞ [`pathway-gomf_PATHWAY_METADATA.txt`](https://www.dropbox.com/s/gfqt86vujnoo7j5/pathway-gomf_PATHWAY_METADATA.txt?dl=1)
- protein-pathway ➞ [`protein-pathway_PATHWAY_METADATA.txt`](https://www.dropbox.com/s/xadtz4c0ab4a7p9/protein-pathway_PATHWAY_METADATA.txt?dl=1)

In [ ]:
node_type = 'pathway'

for edge_type in tqdm(metadata_file_info[node_type]):
    print('\nPROCESSING EDGE TYPE: {}'.format(edge_type))

    # gather vars for processing data
    data = metadata_file_info[node_type][edge_type]['data']
    edge_data_type = metadata_file_info[node_type][edge_type]['instance_idx']
    inst_idx = edge_data_type.split('-').index('instance')

    # get list of nodes to map and the dictionary to use
    # when nodes are of the same type (i.e. gene-gene)
    if (edge_type.split('-')[0] == edge_type.split('-')[1]):
        nodes = set([x for y in data for x in y])
        
        if edge_type.split('-')[0] in master_metadata_dictionary.keys():
            metadata_dictionaries = master_metadata_dictionary[edge_type.split('-')[0]]
            results = metadata_dictionary_mapper(nodes, metadata_dictionaries)
        else:
            results = metadata_api_mapper(list(nodes))
            
    # when nodes are both instances, but different types (i.e. gene-rna)
    elif edge_data_type.split('-')[0] == edge_data_type.split('-')[1]:
        data_res = []

        for node in edge_type.split('-'):
            nodes = set([x[int(edge_type.split('-').index(node))] for x in data])
            
            if node in master_metadata_dictionary:
                metadata_dictionaries = master_metadata_dictionary[node]
                data_res.append(metadata_dictionary_mapper(nodes, metadata_dictionaries))
            else:
                data_res.append(metadata_api_mapper(list(nodes)))
    
        # combine data into single df
        results = pandas.concat(data_res, ignore_index=True)
                
    # when only one node is an instance
    else:
        nodes = set([x[int(inst_idx)] for x in data])
        
        if edge_type.split('-')[int(inst_idx)] in master_metadata_dictionary.keys():
            metadata_dictionaries = master_metadata_dictionary[edge_type.split('-')[int(inst_idx)]]
            results = metadata_dictionary_mapper(nodes, metadata_dictionaries)
        else:
            results = metadata_api_mapper(list(nodes))

    # write data
    results.to_csv(node_data_location + edge_type + '_' + node_type.upper() + '_METADATA.txt', header = True, sep = '\t', index = False)


<br>

***

### Complexes<a class="anchor" id="complex-metadata"></a>

**Data Source Wiki Page:** [Reactome](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#reactome-pathway-database)    

**Output:**    
- chemical-complex ➞ [`chemical-complex_COMPLEX_METADATA.txt`](https://www.dropbox.com/s/mu53u8fv5v0epvf/chemical-complex_COMPLEX_METADATA.txt?dl=1) 
- complex-complex ➞ [`complex-complex_COMPLEX_METADATA.txt`](https://www.dropbox.com/s/y4qt0ne47ix1tqb/complex-complex_COMPLEX_METADATA.txt?dl=1) 
- complex-pathway ➞ [`complex-pathway_COMPLEX_METADATA.txt`](https://www.dropbox.com/s/6n9w0vvxabi7efl/complex-pathway_COMPLEX_METADATA.txt?dl=1) 

In [ ]:
node_type = 'complex'

for edge_type in tqdm(metadata_file_info[node_type]):
    print('\nPROCESSING EDGE TYPE: {}'.format(edge_type))

    # gather vars for processing data
    data = metadata_file_info[node_type][edge_type]['data']
    edge_data_type = metadata_file_info[node_type][edge_type]['instance_idx']
    inst_idx = edge_data_type.split('-').index('instance')

    # get list of nodes to map and the dictionary to use
    # when nodes are of the same type (i.e. gene-gene)
    if (edge_type.split('-')[0] == edge_type.split('-')[1]):
        nodes = set([x for y in data for x in y])
        
        if edge_type.split('-')[0] in master_metadata_dictionary.keys():
            metadata_dictionaries = master_metadata_dictionary[edge_type.split('-')[0]]
            results = metadata_dictionary_mapper(nodes, metadata_dictionaries)
        else:
            results = metadata_api_mapper(list(nodes))
            
    # when nodes are both instances, but different types (i.e. gene-rna)
    elif edge_data_type.split('-')[0] == edge_data_type.split('-')[1]:
        data_res = []

        for node in edge_type.split('-'):
            nodes = set([x[int(edge_type.split('-').index(node))] for x in data])
            
            if node in master_metadata_dictionary:
                metadata_dictionaries = master_metadata_dictionary[node]
                data_res.append(metadata_dictionary_mapper(nodes, metadata_dictionaries))
            else:
                data_res.append(metadata_api_mapper(list(nodes)))
    
        # combine data into single df
        results = pandas.concat(data_res, ignore_index=True)
                
    # when only one node is an instance
    else:
        nodes = set([x[int(inst_idx)] for x in data])
        
        if edge_type.split('-')[int(inst_idx)] in master_metadata_dictionary.keys():
            metadata_dictionaries = master_metadata_dictionary[edge_type.split('-')[int(inst_idx)]]
            results = metadata_dictionary_mapper(nodes, metadata_dictionaries)
        else:
            results = metadata_api_mapper(list(nodes))

    # write data
    results.to_csv(node_data_location + edge_type + '_' + node_type.upper() + '_METADATA.txt', header = True, sep = '\t', index = False)


<br>

***

### Reactions<a class="anchor" id="reaction-metadata"></a>

**Data Source Wiki Page:** [Reactome](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#reactome-pathway-database)   

**Output:**    
- chemical-reaction ➞ [`chemical-reaction_REACTION_METADATA.txt`](https://www.dropbox.com/s/6iztwaxrhrp1f7h/chemical-reaction_REACTION_METADATA.txt?dl=1)
- protein-reaction ➞ [`protein-reaction_REACTION_METADATA.txt`](https://www.dropbox.com/s/92vsuon54w4uq4j/protein-reaction_REACTION_METADATA.txt?dl=1)

In [ ]:
node_type = 'reaction'

for edge_type in tqdm(metadata_file_info[node_type]):
    print('\nPROCESSING EDGE TYPE: {}'.format(edge_type))

    # gather vars for processing data
    data = metadata_file_info[node_type][edge_type]['data']
    edge_data_type = metadata_file_info[node_type][edge_type]['instance_idx']
    inst_idx = edge_data_type.split('-').index('instance')

    # get list of nodes to map and the dictionary to use
    # when nodes are of the same type (i.e. gene-gene)
    if (edge_type.split('-')[0] == edge_type.split('-')[1]):
        nodes = set([x for y in data for x in y])
        
        if edge_type.split('-')[0] in master_metadata_dictionary.keys():
            metadata_dictionaries = master_metadata_dictionary[edge_type.split('-')[0]]
            results = metadata_dictionary_mapper(nodes, metadata_dictionaries)
        else:
            results = metadata_api_mapper(list(nodes))
            
    # when nodes are both instances, but different types (i.e. gene-rna)
    elif edge_data_type.split('-')[0] == edge_data_type.split('-')[1]:
        data_res = []

        for node in edge_type.split('-'):
            nodes = set([x[int(edge_type.split('-').index(node))] for x in data])
            
            if node in master_metadata_dictionary:
                metadata_dictionaries = master_metadata_dictionary[node]
                data_res.append(metadata_dictionary_mapper(nodes, metadata_dictionaries))
            else:
                data_res.append(metadata_api_mapper(list(nodes)))
    
        # combine data into single df
        results = pandas.concat(data_res, ignore_index=True)
                
    # when only one node is an instance
    else:
        nodes = set([x[int(inst_idx)] for x in data])
        
        if edge_type.split('-')[int(inst_idx)] in master_metadata_dictionary.keys():
            metadata_dictionaries = master_metadata_dictionary[edge_type.split('-')[int(inst_idx)]]
            results = metadata_dictionary_mapper(nodes, metadata_dictionaries)
        else:
            results = metadata_api_mapper(list(nodes))

    # write data
    results.to_csv(node_data_location + edge_type + '_' + node_type.upper() + '_METADATA.txt', header = True, sep = '\t', index = False)


<br>

***

### Variants<a class="anchor" id="variant-metadata"></a>

**Data Source Wiki Page:** [ClinVar](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#clinvar)  

**Output:**  
- variant-disease ➞ [`variant-disease_VARIANT_METADATA.txt`](https://www.dropbox.com/s/vj440u5efwdwibl/variant-disease_VARIANT_METADATA.txt?dl=1)  
- variant-gene ➞ [`variant-gene_VARIANT_METADATA.txt`](https://www.dropbox.com/s/geui7nby9h055bc/variant-gene_VARIANT_METADATA.txt?dl=1)  
- variant-phenotype ➞ [`variant-phenotype_VARIANT_METADATA.txt`](https://www.dropbox.com/s/hnocd802detivdd/variant-phenotype_VARIANT_METADATA.txt?dl=1)  

In [ ]:
node_type = 'variant'

for edge_type in tqdm(metadata_file_info[node_type]):
    print('\nPROCESSING EDGE TYPE: {}'.format(edge_type))

    # gather vars for processing data
    data = metadata_file_info[node_type][edge_type]['data']
    edge_data_type = metadata_file_info[node_type][edge_type]['instance_idx']
    inst_idx = edge_data_type.split('-').index('instance')

    # get list of nodes to map and the dictionary to use
    # when nodes are of the same type (i.e. gene-gene)
    if (edge_type.split('-')[0] == edge_type.split('-')[1]):
        nodes = set([x for y in data for x in y])
        
        if edge_type.split('-')[0] in master_metadata_dictionary.keys():
            metadata_dictionaries = master_metadata_dictionary[edge_type.split('-')[0]]
            results = metadata_dictionary_mapper(nodes, metadata_dictionaries)
        else:
            results = metadata_api_mapper(list(nodes))
            
    # when nodes are both instances, but different types (i.e. gene-rna)
    elif edge_data_type.split('-')[0] == edge_data_type.split('-')[1]:
        data_res = []

        for node in edge_type.split('-'):
            nodes = set([x[int(edge_type.split('-').index(node))] for x in data])
            
            if node in master_metadata_dictionary:
                metadata_dictionaries = master_metadata_dictionary[node]
                data_res.append(metadata_dictionary_mapper(nodes, metadata_dictionaries))
            else:
                data_res.append(metadata_api_mapper(list(nodes)))
    
        # combine data into single df
        results = pandas.concat(data_res, ignore_index=True)
                
    # when only one node is an instance
    else:
        nodes = set([x[int(inst_idx)] for x in data])
        
        if edge_type.split('-')[int(inst_idx)] in master_metadata_dictionary.keys():
            metadata_dictionaries = master_metadata_dictionary[edge_type.split('-')[int(inst_idx)]]
            results = metadata_dictionary_mapper(nodes, metadata_dictionaries)
        else:
            results = metadata_api_mapper(list(nodes))

    # write data
    results.to_csv(node_data_location + edge_type + '_' + node_type.upper() + '_METADATA.txt', header = True, sep = '\t', index = False)



<br>

***
***

```
@misc{callahan_tj_2019_3401437,
  author       = {Callahan, TJ},
  title        = {PheKnowLator},
  month        = mar,
  year         = 2019,
  doi          = {10.5281/zenodo.3401437},
  url          = {https://doi.org/10.5281/zenodo.3401437}
}
```